# 0. Libraries and importing the data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import itertools
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from group_lasso import GroupLasso
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

## Preprocessing version 1

We import the data from the final EDA dataset:

In [8]:
df_v1 = pd.read_csv('2_preprocessed_datasets/v1_prepr_ohe-listwise_deletion-no_num_supm-with_num_rooms_outliers.csv')
df_v1

,id,num_rooms,num_baths,square_meters,year_built,is_furnished,has_pool,num_crimes,has_ac,accepts_pets,...,floor_1,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9
0,1562,4.0,1.0,133.0,1960.0,False,False,4.0,False,False,...,False,False,False,False,False,False,False,True,False,False
1,5934,4.0,2.0,77.0,1987.0,True,True,0.0,False,False,...,True,False,False,False,False,False,False,False,False,False
2,3509,1.0,1.0,59.0,1969.0,True,False,0.0,False,False,...,False,False,False,False,False,True,False,False,False,False
3,5867,3.0,3.0,101.0,2014.0,False,False,9.0,False,True,...,False,False,False,False,False,False,False,True,False,False
4,169,3.0,2.0,115.0,2018.0,False,False,0.0,False,True,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4619,4427,3.0,2.0,125.0,1996.0,False,False,4.0,True,False,...,False,True,False,False,False,False,False,False,False,False
4620,467,4.0,1.0,157.0,1992.0,True,False,0.0,False,True,...,False,False,False,False,False,False,True,False,False,False
4621,6266,3.0,3.0,62.0,1979.0,True,True,0.0,True,True,...,False,False,True,False,False,False,False,False,False,False
4622,5391,4.0,1.0,89.0,2015.0,True,False,0.0,True,False,...,False,False,False,False,False,False,False,False,False,True


In [9]:
df_v1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4624 entries, 0 to 4623
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                4624 non-null   int64  
 1   num_rooms                         4624 non-null   float64
 2   num_baths                         4624 non-null   float64
 3   square_meters                     4624 non-null   float64
 4   year_built                        4624 non-null   float64
 5   is_furnished                      4624 non-null   bool   
 6   has_pool                          4624 non-null   bool   
 7   num_crimes                        4624 non-null   float64
 8   has_ac                            4624 non-null   bool   
 9   accepts_pets                      4624 non-null   bool   
 10  price                             4624 non-null   int64  
 11  orientation_east                  4624 non-null   bool   
 12  orient

## Preprocessing version 2

In [10]:
df_v2 = pd.read_csv('2_preprocessed_datasets/v2_prepr_ohe_with_na_dummies-no_num_supm-with_num_rooms_outliers.csv')
df_v2

,id,num_rooms,num_baths,square_meters,year_built,is_furnished,has_pool,num_crimes,has_ac,accepts_pets,...,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floor_nan
0,9255,1.0,1.0,NaN,1956.0,False,False,2.0,True,True,...,False,False,True,False,False,False,False,False,False,False
1,1562,4.0,1.0,133.0,1960.0,False,False,4.0,False,False,...,False,False,False,False,False,False,True,False,False,False
2,1671,2.0,3.0,137.0,2000.0,False,True,0.0,False,False,...,False,False,False,False,False,False,False,False,False,False
3,6088,1.0,2.0,41.0,2002.0,False,True,5.0,False,False,...,False,False,False,False,False,True,False,False,False,False
4,6670,2.0,1.0,70.0,1979.0,True,False,0.0,False,True,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,5735,2.0,NaN,95.0,2021.0,False,False,0.0,False,False,...,False,False,False,False,False,False,False,False,True,False
7996,5192,4.0,3.0,166.0,1995.0,False,False,9.0,False,False,...,False,False,False,False,False,False,False,False,True,False
7997,5391,4.0,1.0,89.0,2015.0,True,False,0.0,True,False,...,False,False,False,False,False,False,False,False,True,False
7998,861,4.0,2.0,167.0,1977.0,False,True,0.0,True,False,...,False,False,False,False,False,False,False,True,False,False


In [11]:
df_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 38 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                8000 non-null   int64  
 1   num_rooms                         8000 non-null   float64
 2   num_baths                         7840 non-null   float64
 3   square_meters                     7845 non-null   float64
 4   year_built                        7830 non-null   float64
 5   is_furnished                      7835 non-null   object 
 6   has_pool                          7844 non-null   object 
 7   num_crimes                        7840 non-null   float64
 8   has_ac                            7831 non-null   object 
 9   accepts_pets                      7845 non-null   object 
 10  price                             8000 non-null   int64  
 11  orientation_east                  8000 non-null   bool   
 12  orient

## Preprocessing version 3

In [12]:
df_v3 = pd.read_csv('2_preprocessed_datasets/v3_prepr-no_ohe-no_num_supm-with_num_rooms_outliers.csv')

In [13]:
# Convert the 'floor' column to string, removing the decimal point but keeping NaN values
df_v3['floor'] = df_v3['floor'].apply(lambda x: str(int(x)) if pd.notnull(x) else x)

In [14]:
df_v3

,id,num_rooms,num_baths,square_meters,orientation,year_built,is_furnished,has_pool,neighborhood,num_crimes,has_ac,accepts_pets,price,floor
0,9255,1.0,1.0,NaN,NaN,1956.0,False,False,Sant Martí,2.0,True,True,1096,3
1,1562,4.0,1.0,133.0,west,1960.0,False,False,Sants,4.0,False,False,1396,7
2,1671,2.0,3.0,137.0,NaN,2000.0,False,True,Eixample,0.0,False,False,1263,1
3,6088,1.0,2.0,41.0,NaN,2002.0,False,True,Sants,5.0,False,False,1290,6
4,6670,2.0,1.0,70.0,NaN,1979.0,True,False,Gràcia,0.0,False,True,962,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,5735,2.0,NaN,95.0,south,2021.0,False,False,Sant Martí,0.0,False,False,1066,9
7996,5192,4.0,3.0,166.0,NaN,1995.0,False,False,Nou Barris,9.0,False,False,1255,9
7997,5391,4.0,1.0,89.0,east,2015.0,True,False,Sant Martí,0.0,True,False,1220,9
7998,861,4.0,2.0,167.0,west,1977.0,False,True,Gràcia,0.0,True,False,1547,8


In [15]:
print(df_v3.isin([np.nan]).sum())

id                  0
num_rooms           0
num_baths         160
square_meters     155
orientation      2341
year_built        170
is_furnished      165
has_pool          156
neighborhood      165
num_crimes        160
has_ac            169
accepts_pets      155
price               0
floor             149
dtype: int64


All good for applying scikit-learn's imputers!

In [16]:
df_v3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             8000 non-null   int64  
 1   num_rooms      8000 non-null   float64
 2   num_baths      7840 non-null   float64
 3   square_meters  7845 non-null   float64
 4   orientation    5659 non-null   object 
 5   year_built     7830 non-null   float64
 6   is_furnished   7835 non-null   object 
 7   has_pool       7844 non-null   object 
 8   neighborhood   7835 non-null   object 
 9   num_crimes     7840 non-null   float64
 10  has_ac         7831 non-null   object 
 11  accepts_pets   7845 non-null   object 
 12  price          8000 non-null   int64  
 13  floor          7851 non-null   object 
dtypes: float64(5), int64(2), object(7)
memory usage: 875.1+ KB


All categorical variables are stored as `object`, while the numerical variables are stored as floats or integers.

## Preprocessing version 4

In [4]:
df_v4 = pd.read_csv('2_preprocessed_datasets/v4_prepr-no_ohe-no_num_supm-no_num_rooms_outliers.csv')

In [7]:
# Convert the 'floor' column to string, removing the decimal point but keeping NaN values
df_v4['floor'] = df_v4['floor'].apply(lambda x: str(int(x)) if pd.notnull(x) else x)
df_v4

,id,num_rooms,num_baths,square_meters,orientation,year_built,is_furnished,has_pool,neighborhood,num_crimes,has_ac,accepts_pets,price,floor
0,9255,1.0,1.0,NaN,NaN,1956.0,False,False,Sant Martí,2.0,True,True,1096,3
1,1562,4.0,1.0,133.0,west,1960.0,False,False,Sants,4.0,False,False,1396,7
2,1671,2.0,3.0,137.0,NaN,2000.0,False,True,Eixample,0.0,False,False,1263,1
3,6088,1.0,2.0,41.0,NaN,2002.0,False,True,Sants,5.0,False,False,1290,6
4,6670,2.0,1.0,70.0,NaN,1979.0,True,False,Gràcia,0.0,False,True,962,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,5735,2.0,NaN,95.0,south,2021.0,False,False,Sant Martí,0.0,False,False,1066,9
7996,5192,4.0,3.0,166.0,NaN,1995.0,False,False,Nou Barris,9.0,False,False,1255,9
7997,5391,4.0,1.0,89.0,east,2015.0,True,False,Sant Martí,0.0,True,False,1220,9
7998,861,4.0,2.0,167.0,west,1977.0,False,True,Gràcia,0.0,True,False,1547,8


In [6]:
df_v4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             8000 non-null   int64  
 1   num_rooms      8000 non-null   float64
 2   num_baths      7840 non-null   float64
 3   square_meters  7845 non-null   float64
 4   orientation    5659 non-null   object 
 5   year_built     7830 non-null   float64
 6   is_furnished   7835 non-null   object 
 7   has_pool       7844 non-null   object 
 8   neighborhood   7835 non-null   object 
 9   num_crimes     7840 non-null   float64
 10  has_ac         7831 non-null   object 
 11  accepts_pets   7845 non-null   object 
 12  price          8000 non-null   int64  
 13  floor          7851 non-null   object 
dtypes: float64(5), int64(2), object(7)
memory usage: 875.1+ KB


## Utilities

### For preprocessing v1, v2

In [17]:
# Lists to store the features
floor_dummies = [f'floor_{str(x)}' for x in range(1, 11)]
neighborhood_dummies = ['neighborhood_Ciutat Vella', 'neighborhood_Eixample', 'neighborhood_Gràcia', 'neighborhood_Horta', 
                        'neighborhood_Les Cors', 'neighborhood_Nou Barris',
                        'neighborhood_Sant Andreu', 'neighborhood_Sant Martí', 
                        'neighborhood_Sants', 'neighborhood_Sarrià-Sant Gervasi']
orientation_dummies = ['orientation_east', 'orientation_north', 'orientation_south', 'orientation_west']

categorical_features = ['is_furnished', 'has_pool', 'has_ac','accepts_pets', 
                            orientation_dummies, neighborhood_dummies,floor_dummies]

numerical_features = ['num_rooms',
 'num_baths',
 'square_meters',
 'year_built',
 'num_crimes']

features = numerical_features + categorical_features
features

['num_rooms',
 'num_baths',
 'square_meters',
 'year_built',
 'num_crimes',
 'is_furnished',
 'has_pool',
 'has_ac',
 'accepts_pets',
 ['orientation_east',
  'orientation_north',
  'orientation_south',
  'orientation_west'],
 ['neighborhood_Ciutat Vella',
  'neighborhood_Eixample',
  'neighborhood_Gràcia',
  'neighborhood_Horta',
  'neighborhood_Les Cors',
  'neighborhood_Nou Barris',
  'neighborhood_Sant Andreu',
  'neighborhood_Sant Martí',
  'neighborhood_Sants',
  'neighborhood_Sarrià-Sant Gervasi'],
 ['floor_1',
  'floor_2',
  'floor_3',
  'floor_4',
  'floor_5',
  'floor_6',
  'floor_7',
  'floor_8',
  'floor_9',
  'floor_10']]

Features with k-1 dummies:

In [18]:
floor_dummies_incomplete = [f'floor_{str(x)}' for x in range(2, 11)]
neighborhood_dummies_incomplete = ['neighborhood_Eixample', 'neighborhood_Gràcia', 'neighborhood_Horta', 
                        'neighborhood_Les Cors', 'neighborhood_Nou Barris',
                        'neighborhood_Sant Andreu', 'neighborhood_Sant Martí', 
                        'neighborhood_Sants', 'neighborhood_Sarrià-Sant Gervasi']
orientation_dummies_incomplete = ['orientation_north', 'orientation_south', 'orientation_west']

categorical_features_incomplete = ['is_furnished', 'has_pool', 'has_ac','accepts_pets', 
                            orientation_dummies_incomplete, neighborhood_dummies_incomplete,floor_dummies_incomplete]

features_incomplete = numerical_features + categorical_features_incomplete
features_incomplete

['num_rooms',
 'num_baths',
 'square_meters',
 'year_built',
 'num_crimes',
 'is_furnished',
 'has_pool',
 'has_ac',
 'accepts_pets',
 ['orientation_north', 'orientation_south', 'orientation_west'],
 ['neighborhood_Eixample',
  'neighborhood_Gràcia',
  'neighborhood_Horta',
  'neighborhood_Les Cors',
  'neighborhood_Nou Barris',
  'neighborhood_Sant Andreu',
  'neighborhood_Sant Martí',
  'neighborhood_Sants',
  'neighborhood_Sarrià-Sant Gervasi'],
 ['floor_2',
  'floor_3',
  'floor_4',
  'floor_5',
  'floor_6',
  'floor_7',
  'floor_8',
  'floor_9',
  'floor_10']]

### For preprocessing v3, v4

In [8]:
categorical_cols_v3 = ['orientation', 'neighborhood', 'floor']
boolean_cols_v3 = ['is_furnished', 'has_pool', 'has_ac', 'accepts_pets']
numerical_cols_v3 = ['num_rooms', 'num_baths', 'square_meters', 'year_built', 'num_crimes']

features_v3 = numerical_cols_v3 + boolean_cols_v3 + categorical_cols_v3
print(features_v3)

['num_rooms', 'num_baths', 'square_meters', 'year_built', 'num_crimes', 'is_furnished', 'has_pool', 'has_ac', 'accepts_pets', 'orientation', 'neighborhood', 'floor']


### Functions

Function that flattens a list (i.e., converts a nested list into a 1-D list):

In [20]:
def flatten(lst):
    return list(itertools.chain(*[x if isinstance(x, list) else [x] for x in lst]))

# 3. Feature selection and evaluation
- Apply wrapper methods? Forward Feature Selection, Backward Feature Elimination and/or Sequential Floating.
- Apply regularization techniques? i.e., LASSO, Ridge, etc. They are embedded methods, which involve building and evaluating models for different feature subsets (as well as wrapper methods), but their feature selection process happens at the same time as their model fitting step.

Predict values in the validation dataset(s) and check the accuracy (in this case, through the MSE) of the prediction. Lower MSE indicates better predictions. Go back to some steps in 2 onwards for improving the accuracy of the model. When you want to improve your model, you can either:
- Create new features. This is relevant even if you have considered all of the data you had. For example, if you have the data of an observation, the model may not consider this information adequately. On the other hand, if instead you trained the model with a feature which is the day of the week, that may be relevant because of seasonality.
- Try different models.
- Ensemble models: for instance, if the random forest considers very good the trend and a decision tree the seasonality, you can combine both.
- Hyperparameters tuning.

Things to do in this section:
1. Split the data.
2. Treat missing values (if there is no listwise deletion).
3. Treat outliers.
4. Standardize. Apply same standardization to validation data.
5. Train model for each fold, with each possible combination of regressors.
6. Evaluate model through MSE.
7. Construct dataframe with the coefficient for each variable if it has been included in the model (NaN otherwise), the fold number, the number of regressors included (counting all of the dummy variables individually) and the MSE.

## Function 1: combination of features (considering OHE variables as a unit; exclude one dummy before to prevent multicollinearity!) -> flatten list of features -> standardization -> linear regression -> cross-validation

In [21]:
# Function to perform linear regressions for each subset of features, and with
# cross-validation

def lin_reg_cv_v5(df: pd.DataFrame, features: list, target: str, n_folds: int = 5):

    def flatten_combo(combo):
        """
        Flattens the combination by including all variables from nested lists
        when one element of a list is selected.
        """
        return list(itertools.chain(*[x if isinstance(x, list) else [x] for x in combo]))

    results_list = []  # To store all results we want for our data frame
    combo_counter = 0  # Initialize a counter for the combinations

    # We create a loop which will consider each possible combination of features
    # from 1 regressor up to the total number of regressors
    for r in range(1, len(features) + 1):
        for combo in itertools.combinations(features, r):
            
            # We add 1 to the counter each time we consider a combination
            combo_counter += 1

            # We flatten the list of the resulting combination of features
            flat_combo = flatten_combo(combo)

            # We create a pipeline where, first, the training data will be standardized
            # and, then, the same standardization will be applied to the test data.
            # Second, the pipeline will fit a linear regression onto the training data.
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('linreg', LinearRegression(fit_intercept = True))
            ])

            # Perform cross-validation: on each fold, it will scale and fit the training
            # data, and compute the negative mean squared error (so a higher value, i.e.,
            # more negative, is better).
            cv_results = cross_validate(estimator=pipeline, 
                                        X=df[flat_combo], y=df[target], cv=n_folds, 
                                        scoring='neg_mean_squared_error',
                                        return_estimator=True)

            neg_mse = cv_results['test_score']  # Negative MSE scores
            estimators = cv_results['estimator']  # List of fitted estimators for each fold

            # Loop through each fold result
            for index, (score, estimator) in enumerate(zip(neg_mse, estimators)):
                linreg = estimator.named_steps['linreg']
                coefs = linreg.coef_
                intercept = linreg.intercept_

                # Create a dictionary to store the results for this fold
                result_dict = {
                    'Fold': index + 1,
                    'Combination ID': combo_counter,
                    'Negative MSE': score,
                    'Intercept': intercept
                }

                # Assign coefficients to the corresponding features
                for i, feature in enumerate(flat_combo):
                    result_dict[feature] = coefs[i]

                # For features not in the current combination, assign NaN
                for feature_group in features:
                    # Flatten the feature group (handle nested lists)
                    feature_list = flatten_combo([feature_group])
                    
                    # If the feature group was not selected, assign NaN to all its members
                    if not any(f in flat_combo for f in feature_list):
                        for feature in feature_list:
                            result_dict[feature] = float('nan')

                # Append the result to the list
                results_list.append(result_dict)

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results_list)

    return results_df

References:
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
- https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html
- https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
- https://www.geeksforgeeks.org/what-is-exactly-sklearnpipelinepipeline/
- https://docs.python.org/3/library/itertools.html#itertools.combinations

### Combinations

Number of combinations in this function for 12 features (considering each group of dummies that come from one variable as a feature):

In [22]:
combo_list = []

for r in range(1, len(features) + 1):
    for combo in itertools.combinations(features, r):
        combo_list.append(list(combo))

print(len(combo_list))

4095


Which matches to the sum of combinations of assigning 12 regressors to k groups, from k = 1 to k = 12!

In [23]:
comb = 0

for k in range(1, 12+1):
    num_comb = math.comb(12, k)
    comb += num_comb

print(comb)

4095


So if we do 5 folds, there will be 4095 * 5 = 20475 different fitted regressions!

### Training with function 1

First, we drop a dummy from each of the categorical variables that we have OHE, in order to prevent multicollinearity:

In [24]:
floor_dummies_f1 = [f'floor_{str(x)}' for x in range(2, 11)]
neighborhood_dummies_f1 = ['neighborhood_Eixample', 'neighborhood_Gràcia', 'neighborhood_Horta', 
                        'neighborhood_Les Cors', 'neighborhood_Nou Barris',
                        'neighborhood_Sant Andreu', 'neighborhood_Sant Martí', 
                        'neighborhood_Sants', 'neighborhood_Sarrià-Sant Gervasi']
orientation_dummies_f1 = ['orientation_north', 'orientation_south', 'orientation_west']

categorical_features_f1 = ['is_furnished', 'has_pool', 'has_ac','accepts_pets', 
                            orientation_dummies_f1, neighborhood_dummies_f1,floor_dummies_f1]

features_f1 = numerical_features + categorical_features_f1
features_f1

['num_rooms',
 'num_baths',
 'square_meters',
 'year_built',
 'num_crimes',
 'is_furnished',
 'has_pool',
 'has_ac',
 'accepts_pets',
 ['orientation_north', 'orientation_south', 'orientation_west'],
 ['neighborhood_Eixample',
  'neighborhood_Gràcia',
  'neighborhood_Horta',
  'neighborhood_Les Cors',
  'neighborhood_Nou Barris',
  'neighborhood_Sant Andreu',
  'neighborhood_Sant Martí',
  'neighborhood_Sants',
  'neighborhood_Sarrià-Sant Gervasi'],
 ['floor_2',
  'floor_3',
  'floor_4',
  'floor_5',
  'floor_6',
  'floor_7',
  'floor_8',
  'floor_9',
  'floor_10']]

Now, we apply the function to dataset v1:

In [25]:
# results_v1_f1 = lin_reg_cv_v5(df = df_v1, features = features_f1, target = 'price', n_folds = 5)
# results_v1_f1.to_csv('3_results_datasets/results_v1_f1.csv')

Approximately 1 minute training.

In [26]:
# results_v1_f1.head()

## Function 2: flatten -> combination of features (automatic exclusion of 1 dummy) -> standardization -> linear regression -> cross-validation
***Function not fine-tuned because it is not computationally feasible to compute all of the possibilities (more than 8 billion combinations)!***

### Combinations

In [27]:
len(flatten(features))

33

In [28]:
# Since there are 33 features overall, the total number of combinations is approx.
# given by:

comb = 0

for k in range(1, len(flatten(features))+1):
    num_comb = math.comb(len(flatten(features)), k)
    comb += num_comb

print(comb)

8589934591


This is probably NOT computationally feasible (more than 8 billion combinations)!!!

## Function 3: features (considering OHE variables as a unit; exclude one dummy before to avoid multicollinearity!) -> flatten list of features -> standardization -> Group LASSO (with alpha parameter tuning) -> cross-validation
Remember that Lasso tends to remove less features than Ridge. 

### Treatment of dummies that come from OHE (ChatGPT)

When dealing with one-hot-encoded categorical variables in Lasso or Ridge, it's important to ensure that the regularization treats each categorical variable as a unit, rather than treating each dummy variable (each column in the one-hot encoding) independently. Otherwise, regularization could shrink or eliminate some dummy variables while keeping others, which can lead to nonsensical results.

Here’s how to handle this situation:

1. **Group Regularization with Lasso/Ridge:**
   - Standard Lasso and Ridge penalize coefficients individually, so if you directly apply them to one-hot-encoded dummy variables, you risk having some dummy variables retained while others are shrunk to zero. This breaks the categorical variable's integrity.
   - To address this, you can use a **grouped version of regularization**, such as:
     - **Group Lasso**: Ensures that either all dummy variables corresponding to a categorical variable are retained or none of them are.
     - **Group Ridge**: Similarly, treats all dummy variables of the same category as a single group when regularizing.

   These methods ensure that regularization applies to the entire set of dummies associated with a categorical variable, keeping the structure of the original categorical variable intact.

2. **Manual Feature Handling (Standard Lasso/Ridge):**
   - If you're using standard Lasso or Ridge without the group version, you may need to manually ensure that either **all or none** of the dummies corresponding to a single categorical variable are included in the model.
   - One way to do this is by **preselecting features** or running regularization only on non-categorical variables, and then adding all the dummy variables of any selected categorical variable manually.

3. **MSE Calculation**:
   - When evaluating the MSE during cross-validation or after fitting the model, ensure that all dummies for a categorical variable are either included or excluded as a group (if you're not using group regularization). This gives you a meaningful prediction that preserves the categorical nature.

In your case, since you are working with feature combinations and treating one-hot-encoded categories as a single feature (from previous discussions), it would make sense to use **Group Lasso** or manually enforce the inclusion of entire dummy groups when checking the MSE.

***On whether to drorp one dummy***:

Yes, you should typically drop one dummy variable when incorporating one-hot-encoded categorical variables into a linear regression model, including Lasso and Ridge regularization. This prevents the problem of perfect multicollinearity, which occurs when the dummy variables are perfectly linearly dependent (i.e., their sum adds up to one).
Why Drop One Dummy?

1. In a one-hot encoding, each category of a variable is represented by a binary column (dummy variable). If you include all dummies, the presence or absence of each category can be perfectly predicted from the others, leading to multicollinearity.
   - Dropping one dummy (usually referred to as the reference category) resolves this issue by making the remaining dummies interpretable as deviations from the reference category.

2. How This Applies to Lasso and Ridge:
   - Lasso and Ridge don't explicitly handle multicollinearity the same way as unregularized linear regression. However, if you don't drop one dummy, the regularization might not fully solve the multicollinearity problem since the model could still have trouble assigning meaningful coefficients to features that are perfectly correlated.
   - By dropping one dummy, you avoid perfect multicollinearity and ensure that your model remains interpretable and stable, whether you're using Lasso, Ridge, or ordinary linear regression.

3. Workflow for One-Hot Encoding with Regularization:

   - One-Hot Encode the Categorical Variables.
   - Drop one dummy from each categorical variable to avoid multicollinearity.
   - Apply Lasso or Ridge to the resulting dataset, ensuring that categorical variables are treated as a unit (using Group Lasso/Ridge or manually ensuring inclusion/exclusion of all dummies for a given category).
   - Check for MSE and proceed with selecting the best features.

By dropping one dummy, you also maintain interpretability because the remaining coefficients for the dummies can be interpreted relative to the reference category.

References:
- https://stats.stackexchange.com/questions/214325/why-use-group-lasso-instead-of-lasso

### Steps to implement Group Lasso (ChatGPT)

To implement **Group Lasso** in Python, you can use the `GroupLasso` class from the `group-lasso` package, which extends the Lasso regularization to treat groups of features (such as one-hot-encoded dummy variables for categorical features) as a unit. Here’s how you can do it:

1. **Install the `group-lasso` package:**
   ```bash
   pip install group-lasso
   ```

2. **Prepare your data:**
   - You need to group the features, which means creating an array that indicates which features belong to the same group (for example, all dummy variables from the same categorical feature would belong to one group).

3. **Define and apply the Group Lasso model:**

Here's a simple example showing how to implement **Group Lasso** for a dataset with one-hot-encoded categorical variables.

In [29]:
# Example data
X = pd.DataFrame({
    'age': [25, 32, 47, 51, 62, 23, 36, 44, 52, 48],
    'income': [50000, 60000, 120000, 100000, 90000, 40000, 75000, 110000, 85000, 95000],
    'gender_male': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
    'gender_female': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    'region_A': [1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    'region_B': [0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
    'region_C': [0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
})

y = np.array([300000, 350000, 500000, 450000, 600000, 200000, 400000, 550000, 500000, 480000])

# Group array, defining which features are part of the same group
# 'age' and 'income' are in separate groups, 'gender_*' is one group, and 'region_*' is another group
groups = [1, 2, 3, 3, 4, 4, 4]  # Group '1' for 'age', '2' for 'income', '3' for 'gender', '4' for 'region'

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the Group Lasso model
group_lasso = GroupLasso(
    groups=groups,  # Grouping array
    group_reg=0.1,  # Regularization strength for groups (default is 0.05)
    l1_reg=0.1,     # Lasso regularization hyperparameter (alpha)
    fit_intercept=True,
    scale_reg="group_size",  # Scales penalty by group size (optional)
    supress_warning = True
)

# Fit the model
group_lasso.fit(X_train_scaled, y_train)

# Predict and calculate MSE
y_pred = group_lasso.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Coefficients: {group_lasso.coef_}')

Mean Squared Error: 3012451088.6857295
Coefficients: [[ 42881.9392877 ]
 [ 60469.30799274]
 [  8070.40757293]
 [ -8070.40757293]
 [ -5453.00239856]
 [ 28470.30292609]
 [-22136.49419348]]


Explanation:

1. **Data**: 
   - `X` is a dataframe containing numerical and one-hot-encoded categorical variables.
   - `groups` specifies how features are grouped. In this example:
     - `age` and `income` are in their own groups.
     - `gender_male` and `gender_female` are in group 3.
     - `region_A`, `region_B`, and `region_C` are in group 4.

2. **Scaling**: The features are standardized before applying Group Lasso since regularization techniques like Lasso and Ridge work better when features are on the same scale.

3. **Group Lasso Model**:
   - `group_reg` controls the regularization strength for the groups (larger values shrink entire groups of features).
   - `l1_reg` controls the Lasso penalty.
   - `groups` defines the group structure, ensuring categorical variables are treated as a unit.

4. **Output**:
   - The model returns the fitted coefficients, and you can calculate the MSE on the test data.

Notes:
- You should adjust the regularization strengths (`group_reg` and `l1_reg`) based on your dataset and desired level of feature selection.
- If you want to treat one-hot-encoded categorical variables as a unit, the group structure should include all dummy variables for a category in the same group.

This method ensures that all dummy variables for a categorical feature are either kept together or removed as a whole during the regularization process.

References:
- https://group-lasso.readthedocs.io/en/latest/
- https://group-lasso.readthedocs.io/en/latest/auto_examples/example_sparse_group_lasso.html
- API reference of GroupLasso: https://group-lasso.readthedocs.io/en/latest/api_reference.html

### Integrating `GroupLasso` in the `scikit-learn` pipeline

Finally, it is also possible to integrate `GroupLasso` within the scikit-learn pipeline. See an example below, which yields the same results as the example above.

In [30]:
# Example dataset
X = pd.DataFrame({
    'age': [25, 32, 47, 51, 62, 23, 36, 44, 52, 48],
    'income': [50000, 60000, 120000, 100000, 90000, 40000, 75000, 110000, 85000, 95000],
    'gender_male': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
    'gender_female': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    'region_A': [1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    'region_B': [0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
    'region_C': [0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
})

y = np.array([300000, 350000, 500000, 450000, 600000, 200000, 400000, 550000, 500000, 480000])

# Group array, defining which features are part of the same group
# 'age' and 'income' are in separate groups, 'gender_*' is one group, and 'region_*' is another group
groups = [1, 2, 3, 3, 4, 4, 4]  # Group '1' for 'age', '2' for 'income', '3' for 'gender', '4' for 'region'

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline for preprocessing and modeling
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('group_lasso', GroupLasso(groups=groups, group_reg=0.1, l1_reg=0.1, fit_intercept=True, supress_warning=True))
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Coefficients: {pipeline.named_steps["group_lasso"].coef_}')
print(f'Intercept: {pipeline.named_steps["group_lasso"].intercept_}')

for i in range(len(pipeline.named_steps["group_lasso"].coef_)):
    print('Coefficient number', i + 1, ':', pipeline.named_steps["group_lasso"].coef_[i][0])

Mean Squared Error: 3012412897.978672
Coefficients: [[ 42882.43237837]
 [ 60468.91167671]
 [  8070.38649143]
 [ -8070.38649143]
 [ -5453.14774949]
 [ 28470.06047625]
 [-22136.51700127]]
Intercept: [435000.86541208]
Coefficient number 1 : 42882.432378368525
Coefficient number 2 : 60468.911676710384
Coefficient number 3 : 8070.386491427103
Coefficient number 4 : -8070.386491427103
Coefficient number 5 : -5453.147749491385
Coefficient number 6 : 28470.060476253948
Coefficient number 7 : -22136.517001267865


### Adding KNN imputer in the pipeline

Finally, it is also possible to integrate the KNN imputer in the pipeline, after standardization but before Group Lasso:

In [31]:
# Example dataset with missing values (NaN)
X = pd.DataFrame({
    'age': [25, 32, np.nan, 51, 62, 23, np.nan, 44, 52, 48],
    'income': [50000, 60000, 120000, np.nan, 90000, 40000, 75000, np.nan, 85000, 95000],
    'gender_male': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
    'gender_female': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
    'region_A': [1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    'region_B': [0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
    'region_C': [0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
})

y = np.array([300000, 350000, 500000, 450000, 600000, 200000, 400000, 550000, 500000, 480000])

# Group array, defining which features are part of the same group
groups = [1, 2, 3, 3, 4, 4, 4]  # Group '1' for 'age', '2' for 'income', '3' for 'gender', '4' for 'region'

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline for preprocessing and modeling
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('imputer', KNNImputer(n_neighbors=3)),  # Impute missing values using KNN
    ('group_lasso', GroupLasso(groups=groups, group_reg=0.1, l1_reg=0.1, fit_intercept=True, scale_reg="group_size", supress_warning=True))
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Coefficients: {pipeline.named_steps["group_lasso"].coef_}')

Mean Squared Error: 13663955712.572994
Coefficients: [[ 19539.83764285]
 [ 95027.61005906]
 [ -3700.95143226]
 [  3700.95143226]
 [ -8145.32673929]
 [ 44530.73101544]
 [-35076.07732598]]


/home/pablo/miniconda3/envs/py312/lib/python3.12/site-packages/group_lasso/_fista.py:114: ConvergenceWarning: The FISTA iterations did not converge to a sufficient minimum.
You used subsampling then this is expected, otherwise, try increasing the number of iterations or decreasing the tolerance.
  warnings.warn(


Key Steps:
1. StandardScaler: First, the features are standardized.
    - Standardization is done before KNN Imputation because the KNN algorithm performs better when features are on the same scale (it uses distance to find the nearest neighbors).
2. KNNImputer: Missing values are imputed using K-nearest neighbors based on the scaled data. The parameter n_neighbors=3 determines how many neighbors are considered for imputation.
3. GroupLasso: After imputation, the GroupLasso model is fitted, with the grouped features as described previously.

Why This Order?
- Standardization before imputation ensures the KNN algorithm can work effectively, as distance-based imputation methods are sensitive to feature scales.
- Imputation after standardization fills in missing values with data that maintains the relationships between features.
- Finally, Group Lasso operates on the fully imputed dataset, with no missing values, enabling group-wise feature selection.

References of `KNNImputer`:
- API: https://scikit-learn.org/1.5/modules/generated/sklearn.impute.KNNImputer.html
- https://scikit-learn.org/1.5/modules/impute.html. Key info on this imputation:
    - The KNNImputer class provides imputation for filling in missing values using the k-Nearest Neighbors approach. By default, a euclidean distance metric that supports missing values, nan_euclidean_distances, is used to find the nearest neighbors. 
    - Each missing feature is imputed using values from n_neighbors nearest neighbors **that have a value for the feature**.
    - The feature of the neighbors are averaged uniformly or weighted by distance to each neighbor. 
    - If a sample has more than one feature missing, then **the neighbors for that sample can be different depending on the particular feature being imputed**.
    - **When the number of available neighbors is less than n_neighbors and there are no defined distances to the training set, the training set average for that feature is used during imputation**.

In [32]:
imputer = KNNImputer(n_neighbors = 5)
# Note how this is not correct as first we would need to standardize the data
imputer.fit_transform(df_v2)
df_v2

,id,num_rooms,num_baths,square_meters,year_built,is_furnished,has_pool,num_crimes,has_ac,accepts_pets,...,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floor_nan
0,9255,1.0,1.0,NaN,1956.0,False,False,2.0,True,True,...,False,False,True,False,False,False,False,False,False,False
1,1562,4.0,1.0,133.0,1960.0,False,False,4.0,False,False,...,False,False,False,False,False,False,True,False,False,False
2,1671,2.0,3.0,137.0,2000.0,False,True,0.0,False,False,...,False,False,False,False,False,False,False,False,False,False
3,6088,1.0,2.0,41.0,2002.0,False,True,5.0,False,False,...,False,False,False,False,False,True,False,False,False,False
4,6670,2.0,1.0,70.0,1979.0,True,False,0.0,False,True,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,5735,2.0,NaN,95.0,2021.0,False,False,0.0,False,False,...,False,False,False,False,False,False,False,False,True,False
7996,5192,4.0,3.0,166.0,1995.0,False,False,9.0,False,False,...,False,False,False,False,False,False,False,False,True,False
7997,5391,4.0,1.0,89.0,2015.0,True,False,0.0,True,False,...,False,False,False,False,False,False,False,False,True,False
7998,861,4.0,2.0,167.0,1977.0,False,True,0.0,True,False,...,False,False,False,False,False,False,False,True,False,False


Nothing has changed!! Note, as well, that in the case of the dummies we are handling NaNs in the dummies themselves, which complicates things. Maybe a better idea would be to label categories and, after that, apply OHE.

### Definition of the function

In [33]:
def group_lasso(df: pd.DataFrame, features: list, target: str, lasso_groups: list, n_folds: int = 5, max_n_lasso_iter: int = 100):

    results_list = []  # To store all results we want for our data frame
    alpha_counter = 0  # Initialize a counter for the alphas

    # We create the steps of the pipeline where, first, the training data will be standardized
    # and, then, the same standardization will be applied to the test data.
    # Second, the pipeline will impute null values through KNN, if it is desired.
    # Finally, it will perform Group Lasso to the imputed training and test dataset.
    steps = [
        ('scaler', StandardScaler()),  # Standardize features
    ]
    
    # Now, we create a first loop that iterates over different values of the alpha
    # parameter. We generate 50 alphas in log from -10 to 0
    for alpha in np.logspace(-10, 0, 50):

        # We add Group Lasso to the pipeline, with a specific alpha
        steps.append(('group_lasso', 
                    GroupLasso(
                        groups = lasso_groups, 
                        group_reg = 0.05, 
                        l1_reg = alpha, 
                        fit_intercept = True, 
                        n_iter = max_n_lasso_iter,
                        scale_reg = "group_size",
                        supress_warning = True
                        )))

        # We add 1 to the counter of alpha each time we consider another alpha
        alpha_counter += 1
        
        # We create the pipeline with the steps specified above
        pipeline = Pipeline(steps)

        # Perform cross-validation: on each fold, it will scale and fit the training
        # data, and compute the negative mean squared error (so a higher value, i.e.,
        # more negative, is better).

        cv_results = cross_validate(
            estimator = pipeline, 
            X = df[features], y = df[target], cv = n_folds, 
            scoring='neg_mean_squared_error',
            return_estimator=True
            )

        neg_mse = cv_results['test_score']  # Negative MSE scores
        estimators = cv_results['estimator']  # List of fitted estimators for each fold
        
        # Loop through each fold result to store the information
        for index, (score, estimator) in enumerate(zip(neg_mse, estimators)):
            # We access the information associated to the group lasso
            group_lasso = estimator.named_steps['group_lasso']
            # We store the coefficients estimated by group lasso
            coefs = group_lasso.coef_
            # We store the intercept estimated by group lasso
            intercept = group_lasso.intercept_[0]

            # Create a dictionary to store the results for this fold
            result_dict = {
                'Fold': index + 1,
                'Alpha_counter': alpha_counter,
                'Alpha': alpha,
                'Negative MSE': score,
                'Intercept': intercept
            }

            # Assign coefficients to the corresponding features
            for i, feature in enumerate(features):
                result_dict[feature] = coefs[i][0]

            # Append the result to the list
            results_list.append(result_dict)


        # Lastly, we remove the last step so that the GroupLasso step does not accumulate
        # with different alphas
        steps.pop()

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results_list)

    return results_df

Possible improvement: making `group_reg` configurable through parameters (just like looping through alphas).

### Training with function 3

First, we define the group array, establishing which features are part of the same group.

In [34]:
# First, we flatten the list of the features with k-1 dummies
features_incomplete_flattened = flatten(features_incomplete)
print(features_incomplete_flattened)

['num_rooms', 'num_baths', 'square_meters', 'year_built', 'num_crimes', 'is_furnished', 'has_pool', 'has_ac', 'accepts_pets', 'orientation_north', 'orientation_south', 'orientation_west', 'neighborhood_Eixample', 'neighborhood_Gràcia', 'neighborhood_Horta', 'neighborhood_Les Cors', 'neighborhood_Nou Barris', 'neighborhood_Sant Andreu', 'neighborhood_Sant Martí', 'neighborhood_Sants', 'neighborhood_Sarrià-Sant Gervasi', 'floor_2', 'floor_3', 'floor_4', 'floor_5', 'floor_6', 'floor_7', 'floor_8', 'floor_9', 'floor_10']


Groups (for `GroupLasso`):

In [35]:
groups_feat_incomp_flat = [
    1,  # 'num_rooms'
    2,  # 'num_baths'
    3,  # 'square_meters'
    4,  # 'year_built'
    5,  # 'num_crimes'
    6,  # 'is_furnished'
    7,  # 'has_pool'
    8,  # 'has_ac'
    9,  # 'accepts_pets'
    
    # Orientation group
    10,  # 'orientation_north'
    10,  # 'orientation_south'
    10,  # 'orientation_west'
    
    # Neighborhood group
    11,  # 'neighborhood_Eixample'
    11,  # 'neighborhood_Gràcia'
    11,  # 'neighborhood_Horta'
    11,  # 'neighborhood_Les Cors'
    11,  # 'neighborhood_Nou Barris'
    11,  # 'neighborhood_Sant Andreu'
    11,  # 'neighborhood_Sant Martí'
    11,  # 'neighborhood_Sants'
    11,  # 'neighborhood_Sarrià-Sant Gervasi'

    # Floor group
    12,  # 'floor_2'
    12,  # 'floor_3'
    12,  # 'floor_4'
    12,  # 'floor_5'
    12,  # 'floor_6'
    12,  # 'floor_7'
    12,  # 'floor_8'
    12,  # 'floor_9'
    12,  # 'floor_10'
]

In [36]:
# results_v1_f3 = group_lasso(
    # df = df_v1, 
    # features = features_incomplete_flattened,
    # target = 'price',
    # lasso_groups = groups_feat_incomp_flat,
    # n_folds = 5,
    # use_knn_imputer = False,
    # my_n_neighbors = 5,
    # max_n_lasso_iter = 500)

# results_v1_f3.to_csv('3_results_datasets/results_v1_f3.csv')

Approx. 1 minute to execute.

In [37]:
# results_v1_f3

### Extended version of function 3 with loop for different values of group regularization penalty

In [38]:
def group_lasso_ext(df: pd.DataFrame, features: list, target: str, lasso_groups: list, n_folds: int = 5, max_n_lasso_iter: int = 100):

    results_list = []  # To store all results we want for our data frame
    alpha_counter = 0  # Initialize a counter for the alphas
    group_reg_parameter_counter = 0 # Initialize a counter for the group regularization parameter

    # We create the steps of the pipeline where, first, the training data will be standardized
    # and, then, the same standardization will be applied to the test data.
    # Second, the pipeline will impute null values through KNN, if it is desired.
    # Finally, it will perform Group Lasso to the imputed training and test dataset.
    steps = [
        ('scaler', StandardScaler()),  # Standardize features
    ]
    
    # Now, we create a first loop that iterates over different values of the alpha
    # parameter. We generate 50 alphas in log from -10 to 0
    for alpha in np.logspace(-10, 0, 50):
    
        # We add 1 to the counter of alpha each time we consider another alpha
        alpha_counter += 1

        # We create a second loop that iterates over different values of the 
        # parameter that penalizes group sparsity
        for group_parameter in np.logspace(-10, 0, 50):

            # We add Group Lasso to the pipeline, with a specific alpha
            steps.append(('group_lasso', 
                        GroupLasso(
                            groups = lasso_groups, 
                            group_reg = group_parameter, 
                            l1_reg = alpha, 
                            fit_intercept = True, 
                            n_iter = max_n_lasso_iter,
                            scale_reg = "group_size",
                            supress_warning = True
                            )))

            # We add 1 to the counter of the group regularization parameter in each iteration
            group_reg_parameter_counter += 1
            
            # We create the pipeline with the steps specified above
            pipeline = Pipeline(steps)

            # Perform cross-validation: on each fold, it will scale and fit the training
            # data, and compute the negative mean squared error (so a higher value, i.e.,
            # more negative, is better).

            cv_results = cross_validate(
                estimator = pipeline, 
                X = df[features], y = df[target], cv = n_folds, 
                scoring='neg_mean_squared_error',
                return_estimator=True
                )

            neg_mse = cv_results['test_score']  # Negative MSE scores
            estimators = cv_results['estimator']  # List of fitted estimators for each fold
            
            # Loop through each fold result to store the information
            for index, (score, estimator) in enumerate(zip(neg_mse, estimators)):
                # We access the information associated to the group lasso
                group_lasso = estimator.named_steps['group_lasso']
                # We store the coefficients estimated by group lasso
                coefs = group_lasso.coef_
                # We store the intercept estimated by group lasso
                intercept = group_lasso.intercept_[0]

                # Create a dictionary to store the results for this fold
                result_dict = {
                    'Fold': index + 1,
                    'Alpha_counter': alpha_counter,
                    'Group_reg_parameter_counter': group_reg_parameter_counter,
                    'Alpha': alpha,
                    'Group_reg_parameter': group_parameter,
                    'Negative MSE': score,
                    'Intercept': intercept
                }

                # Assign coefficients to the corresponding features
                for i, feature in enumerate(features):
                    result_dict[feature] = coefs[i][0]

                # Append the result to the list
                results_list.append(result_dict)
    

            # Lastly, we remove the last step so that the GroupLasso step does not accumulate
            # with different hyperparameters
            steps.pop()

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results_list)

    return results_df

## Function 4: features -> flatten list of features -> standardization -> Group LASSO (with alpha parameter tuning + group regularization penalty tuning) -> cross-validation

### Addressing the issue of KNN Imputer and categorical variables

1. **KNN Imputer with One-Hot Encoded Dummies**:
- The KNN Imputer can be problematic for one-hot encoded categorical variables. The imputer treats all columns as continuous values, so it may impute fractional values for one-hot encoded columns, which is not meaningful (e.g., [0.2, 0.6, 0.2] instead of [0, 1, 0]).
- Solution: Before applying the KNN Imputer, you can either:
    - Keep categorical variables in their original (labeled) form, impute them, and then re-encode them after the imputation.
    - Alternatively, use the `SimpleImputer` with the `most_frequent` strategy for categorical features, and KNN Imputer only for numerical features.
2. **Labeled Categorical Variables and Standardization**:
- Labeled categorical variables (e.g., blue = 1, red = 2, green = 3) are problematic when standardized because they can imply an ordinal relationship (e.g., green > blue), which may not be valid. Standardization will treat them as continuous values, causing distortions in imputation.
- Solution: You should separate categorical and numerical columns and apply different preprocessing strategies to each:
    - Numerical Columns: Standardize and then impute with KNN.
    - Categorical Columns: Use imputation strategies like `SimpleImputer` or use KNN without standardization.

***The most important thing to avoid, in any case, is to impute values including the target variable in the imputation as well!***

### Trying a new approach for combining transformations based on the distinction of numerical and categorical variables

#### A. OHE

In [39]:
# First, we separate categorical and numerical columns. In df_v3, all categorical
# variables are stored as objects
# categorical_cols = df_v3[features_v3].select_dtypes(include=['category', 'object']).columns.tolist()
# numerical_cols = df_v3[features_v3].select_dtypes(include=['float64', 'int64']).columns.tolist()

# print(categorical_cols)
# print(numerical_cols)

This could be handled by specifying which columns you want to consider as categorical and which as numerical, though, which gives more control in this case given that we don't have many features.

In [40]:
# How OHE works in scikit-learn
ohe = OneHotEncoder(handle_unknown = 'error', sparse_output = False)

ohe_array = ohe.fit_transform(df_v3[categorical_cols_v3])

print(ohe.categories_)

[array(['east', 'north', 'south', 'west', nan], dtype=object), array(['Ciutat Vella', 'Eixample', 'Gràcia', 'Horta', 'Les Cors',
       'Nou Barris', 'Sant Andreu', 'Sant Martí', 'Sants',
       'Sarrià-Sant Gervasi', nan], dtype=object), array(['1', '10', '2', '3', '4', '5', '6', '7', '8', '9', nan],
      dtype=object)]


In [41]:
# Get the feature names from OneHotEncoder
ohe_feature_names = ohe.get_feature_names_out(categorical_cols_v3)

# Convert the array back to a DataFrame
ohe_df = pd.DataFrame(ohe_array, columns=ohe_feature_names, index=df_v3.index)
ohe_df

# Now you could concatenate this back with the original dataframe (excluding the original categorical columns)
# df_v3_encoded = pd.concat([df_v3.drop(categorical_cols, axis=1), ohe_df], axis=1)

,orientation_east,orientation_north,orientation_south,orientation_west,orientation_nan,neighborhood_Ciutat Vella,neighborhood_Eixample,neighborhood_Gràcia,neighborhood_Horta,neighborhood_Les Cors,...,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floor_nan
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7996,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7997,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7998,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


OHE of scikit-learn cannot handle null values, as it will consider those as a category by themselves. That is also a reason why imputing must be done *before* encoding. Note, however, that dummy variables are treated again as variables to encode: we must handle that, maybe by excluding them from OHE. For more information, see https://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.OneHotEncoder.html. Probably, the best thing to do is to indicate, manually, which columns are to be considered categorical, which numerical and which boolean.

#### B. Different transformations and imputations with `ColumnTransformer`

Below, we are implementing the separate transformation into a pipeline through `Pipeline` and `ColumnTransformer`.
- Documentation for `ColumnTransformer`: https://scikit-learn.org/1.5/modules/generated/sklearn.compose.ColumnTransformer.html. According to the documentation, "This estimator allows different columns or column subsets of the input to be transformed ***separately*** and **the features generated by each transformer will be concatenated to form a single feature space**. This is **useful for heterogeneous or columnar data, to combine several feature extraction mechanisms or transformations into a single transformer**."
- Documentation for `Pipeline`: https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html. More information and intuition on the implementation in previous functions.

Specifically, we are doing the following:
1. We are creating two lists with tuples inside where we store the specific transformation for each type of variable (standardization for numerical and OHE for categorical).
2. Then, we are inserting (through `insert`: https://docs.python.org/3/tutorial/datastructures.html), in different places of the list that will act as the steps of the preprocessor, the imputing method (KNN for numerical, mode or most frequent value for categorical variables and dummies). Note how we are inserting the tuple in the same way as the `Pipeline` expects. `SimpleImputer` doc https://scikit-learn.org/1.5/modules/generated/sklearn.impute.SimpleImputer.html
3. We create a ColumnTransformer class for being able to apply different transforming and/or imputation strategies to different features.
4. Finally, we create a list where we will store the steps for the pipeline where we will insert Group Lasso.

In [42]:
# Create transformers for categorical and numerical features
numerical_transformer = [
    ('scaler', StandardScaler())  # Standardize numerical features
    ]
    
categorical_transformer = [
    ('onehot', OneHotEncoder(handle_unknown='error', drop = 'first'))  # One-hot encode categorical features, dropping first dummy
]

my_n_neighbors = 5

# Add KNN imputer for numerical features. We append it, so that standardization is
# performed before KNN Imputing
numerical_transformer.append(('knn_imputer', KNNImputer(n_neighbors = my_n_neighbors)))
# Optionally, use SimpleImputer for categorical features instead of KNN. We insert it 
# BEFORE one-hot encoding the categorical variables
categorical_transformer.insert(0, ('simple_imputer', SimpleImputer(strategy='most_frequent')))

# Create the ColumnTransformer to apply different transformations to numerical
# columns, categorical and dummy (boolean) columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(numerical_transformer), numerical_cols_v3),
        ('cat', Pipeline(categorical_transformer), categorical_cols_v3),
        ('bool', SimpleImputer(strategy='most_frequent'), boolean_cols_v3)
    ]
)

# Create steps of the pipeline
steps = [('preprocessor', preprocessor)]

##### Testing with fake data

Now, we can test whether this pipeline with a `ColumnTransformer` works, first with some sample data and then with our data: 

In [43]:
# Sample DataFrame
data = {
    'numerical_1': [1, 2, np.nan, 4],
    'numerical_2': [5, np.nan, 7, 8],
    'categorical_1': ['A', 'B', 'A', np.nan],
    'categorical_2': ['X', np.nan, 'Y', 'Z'],
    'boolean_1': [1, 0, np.nan, 1]
}

df = pd.DataFrame(data)

# Define the column lists
numerical_cols_test = ['numerical_1', 'numerical_2']
categorical_cols_test = ['categorical_1', 'categorical_2']
boolean_cols_test = ['boolean_1']

###############################################################################

# Create transformers for categorical and numerical features
numerical_transformer = [
    ('scaler', StandardScaler())  # Standardize numerical features
    ]
    
categorical_transformer = [
    ('onehot', OneHotEncoder(handle_unknown='error', drop = 'first'))  # One-hot encode categorical features, dropping first dummy
]

my_n_neighbors = 5

# Add KNN imputer for numerical features. We append it, so that standardization is
# performed before KNN Imputing
numerical_transformer.append(('knn_imputer', KNNImputer(n_neighbors = my_n_neighbors)))
# Optionally, use SimpleImputer for categorical features instead of KNN. We insert it 
# BEFORE one-hot encoding the categorical variables
categorical_transformer.insert(0, ('simple_imputer', SimpleImputer(strategy='most_frequent')))

# Create the ColumnTransformer to apply different transformations to numerical
# columns, categorical and dummy (boolean) columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(numerical_transformer), numerical_cols_test),
        ('cat', Pipeline(categorical_transformer), categorical_cols_test),
        ('bool', SimpleImputer(strategy='most_frequent'), boolean_cols_test)
    ]
)

# Create steps of the pipeline
steps = [('preprocessor', preprocessor)]

###############################################################################

# Fit the pipeline to the sample data
pipeline_test = Pipeline(steps)
pipeline_test.fit(df)

# Apply the transformations
transformed_data_test = pipeline_test.transform(df)

# Get feature names after transformations
numerical_feature_names_test = numerical_cols_test  # They remain the same after scaling
categorical_feature_names_test = pipeline_test.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_cols_test)
boolean_feature_names_test = boolean_cols_test  # They remain the same after imputation

# Combine all feature names
all_feature_names_test = list(numerical_feature_names_test) + list(categorical_feature_names_test) + list(boolean_feature_names_test)

# Convert the transformed data to a DataFrame with proper column names
transformed_df_test = pd.DataFrame(transformed_data_test, columns=all_feature_names_test)
display(transformed_df_test)

,numerical_1,numerical_2,categorical_1_B,categorical_2_Y,categorical_2_Z,boolean_1
0,-1.069045e+00,-1.336306e+00,0.0,0.0,0.0,1.0
1,-2.672612e-01,-2.590520e-16,1.0,0.0,0.0,0.0
2,-1.480297e-16,2.672612e-01,0.0,1.0,0.0,1.0
3,1.336306e+00,1.069045e+00,0.0,0.0,1.0,1.0


This first attempt seems to have worked. All the transformations have been done as expected. Now, let's see how it performs with our data.

##### Testing with our data

In [44]:
# Fit the pipeline to the sample data
pipeline = Pipeline(steps)
pipeline.fit(df_v3[features_v3])

# Apply the transformations
transformed_data = pipeline.transform(df_v3[features_v3])

# Get feature names after transformations
numerical_feature_names = numerical_cols_v3  # They remain the same after scaling
categorical_feature_names = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_cols_v3)
boolean_feature_names = boolean_cols_v3  # They remain the same after imputation

# Combine all feature names
all_feature_names = list(numerical_feature_names) + list(boolean_feature_names) + list(categorical_feature_names) 

# Convert the transformed data to a DataFrame with proper column names
transformed_df = pd.DataFrame(transformed_data, columns=all_feature_names)

# We enforce boolean and categorical columns to be interpreted as integers, and numerical as floats
transformed_df[numerical_feature_names] = transformed_df[numerical_feature_names].astype(float)
transformed_df[boolean_feature_names] = transformed_df[boolean_feature_names].astype(int)
transformed_df[categorical_feature_names] = transformed_df[categorical_feature_names].astype(int)

display(transformed_df)

ValueError: A given column is not a column of the dataframe

In [71]:
print(transformed_df.columns)

Index(['num_rooms', 'num_baths', 'square_meters', 'year_built', 'num_crimes',
       'is_furnished', 'has_pool', 'has_ac', 'accepts_pets',
       'orientation_north', 'orientation_south', 'orientation_west',
       'neighborhood_Eixample', 'neighborhood_Gràcia', 'neighborhood_Horta',
       'neighborhood_Les Cors', 'neighborhood_Nou Barris',
       'neighborhood_Sant Andreu', 'neighborhood_Sant Martí',
       'neighborhood_Sants', 'neighborhood_Sarrià-Sant Gervasi', 'floor_10',
       'floor_2', 'floor_3', 'floor_4', 'floor_5', 'floor_6', 'floor_7',
       'floor_8', 'floor_9'],
      dtype='object')


In [97]:
print(transformed_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   num_rooms                         8000 non-null   float64
 1   num_baths                         8000 non-null   float64
 2   square_meters                     8000 non-null   float64
 3   year_built                        8000 non-null   float64
 4   num_crimes                        8000 non-null   float64
 5   is_furnished                      8000 non-null   int64  
 6   has_pool                          8000 non-null   int64  
 7   has_ac                            8000 non-null   int64  
 8   accepts_pets                      8000 non-null   int64  
 9   orientation_north                 8000 non-null   int64  
 10  orientation_south                 8000 non-null   int64  
 11  orientation_west                  8000 non-null   int64  
 12  neighb

Everything seems to work as expected!

#### C. Checking whether `cross_validate` applies the transformations and imputations separately to the training and test dataset
The idea now is to extend the example above and check whether the pipeline works also when applying cross-validation.

In [82]:
# Create transformers for categorical and numerical features
numerical_transformer = [
    ('scaler', StandardScaler())  # Standardize numerical features
    ]
    
categorical_transformer = [
    ('onehot', OneHotEncoder(handle_unknown='error', drop = 'first'))  # One-hot encode categorical features, dropping first dummy
]

my_n_neighbors = 5

# Add KNN imputer for numerical features. We append it, so that standardization is
# performed before KNN Imputing
numerical_transformer.append(('knn_imputer', KNNImputer(n_neighbors = my_n_neighbors)))
# Optionally, use SimpleImputer for categorical features instead of KNN. We insert it 
# BEFORE one-hot encoding the categorical variables
categorical_transformer.insert(0, ('simple_imputer', SimpleImputer(strategy='most_frequent')))

# Create the ColumnTransformer to apply different transformations to numerical
# columns, categorical and dummy (boolean) columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(numerical_transformer), numerical_cols_v3),
        ('cat', Pipeline(categorical_transformer), categorical_cols_v3),
        ('bool', SimpleImputer(strategy='most_frequent'), boolean_cols_v3)
    ]
)

# Create steps of the pipeline
steps = [('preprocessor', preprocessor)]

###############################################################################

# Fit the pipeline to our data
pipeline = Pipeline(steps)

###############################################################################

# We perform cross-validation

target = 'price'
n_folds = 5

cv_results = cross_validate(
                estimator = pipeline, 
                X = df_v3[features_v3], y = df_v3[target], cv = n_folds, 
                scoring='neg_mean_squared_error',
                return_estimator=True
                )

/home/pablo/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/pablo/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pablo/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pablo/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 370, in _score
    response_method = _check_response_method(estimator, self._respons

The error above appears because there is no class in the pipeline with the predict attribute. This should be resolved when adding GroupLasso to the equation.

To ensure that your pipeline handles transformations correctly in the cross-validation process, you can take these steps to verify if each step behaves as expected in avoiding data leakage:

1. **Verify Transformation Steps on Estimators in Each Fold**: Since `cross_validate` with `return_estimator=True` will return fitted estimators for each fold, you can inspect each estimator after cross-validation to confirm the behavior of your pipeline components.

In [83]:
for idx, estimator in enumerate(cv_results['estimator']):
    print(f"Estimator {idx+1}")
    # Check numerical scaler mean and variance for each fold
    print("Scaler mean:", estimator.named_steps['preprocessor'].named_transformers_['num'].named_steps['scaler'].mean_)
    print("Scaler variance:", estimator.named_steps['preprocessor'].named_transformers_['num'].named_steps['scaler'].var_)
    # Check KNN Imputer for fitted values
    print("KNN Imputer statistics:", estimator.named_steps['preprocessor'].named_transformers_['num'].named_steps['knn_imputer'])
    # Check categorical most frequent values
    print("SimpleImputer most frequent:", estimator.named_steps['preprocessor'].named_transformers_['cat'].named_steps['simple_imputer'].statistics_)

Estimator 1
Scaler mean: [   2.73296875    1.99585195  109.06858054 1985.33067348    2.81066411]
Scaler variance: [1.81957256e+01 6.65585857e-01 1.23214362e+03 4.21786601e+02
 1.19069999e+01]
KNN Imputer statistics: KNNImputer()
SimpleImputer most frequent: ['west' 'Sants' '5']
Estimator 2
Scaler mean: [2.85343750e+00 1.98232765e+00 1.09466486e+02 1.98506523e+03
 2.79326846e+00]
Scaler variance: [2.77491444e+01 6.64709182e-01 1.27055657e+03 4.30058095e+02
 1.17955138e+01]
KNN Imputer statistics: KNNImputer()
SimpleImputer most frequent: ['west' 'Sants' '6']
Estimator 3
Scaler mean: [   2.894375      1.9856528   109.18818659 1985.37001595    2.8181963 ]
Scaler variance: [2.92738434e+01 6.66460824e-01 1.25215436e+03 4.22537729e+02
 1.19588212e+01]
KNN Imputer statistics: KNNImputer()
SimpleImputer most frequent: ['west' 'Nou Barris' '5']
Estimator 4
Scaler mean: [   2.92578125    1.99058861  109.06782359 1985.39105431    2.80592734]
Scaler variance: [3.21890228e+01 6.64291710e-01 1.26048

Your output indicates that the pipeline components are working independently within each cross-validation fold, which helps avoid data leakage. Let’s review each part of the output in light of your expectations:

1. **Scaler Mean and Variance**:
   - Each estimator shows different means and variances for the `StandardScaler`, which suggests that the scaling is being calculated separately for each training fold. This is expected and desired, as it means the scaler is not influenced by data in the validation set.
   
2. **KNN Imputer**:
   - Since `KNNImputer()` doesn’t return specific statistics, it simply shows as `KNNImputer()`. However, its presence here indicates it is applied independently in each fold, as expected. The variation in `StandardScaler` parameters across folds implies that KNN Imputation is applied after scaling, which aligns with your specified order.

3. **SimpleImputer (Most Frequent) for Categorical Variables**:
   - The output shows different “most frequent” values in each fold (e.g., "Sants" vs. "Nou Barris" and "5" vs. "6"), which confirms that `SimpleImputer` is computing the mode independently for each training fold. This avoids any leakage of the most frequent categories from the validation set.

4. **SimpleImputer for Boolean Columns**:
   - If boolean columns had specific values imputed, you would expect to see similar changes here as you see for categorical columns, with the most frequent values varying slightly across folds (if they were not specified in your code block).

**Summary**

Everything appears correct:
- **Scaling** is fold-specific, ensuring no leakage.
- **KNN Imputation** respects fold boundaries.
- **Categorical and Boolean Imputation** is fold-specific, confirming independent fitting.

Your cross-validation setup is performing as expected.

---

2. **Ensure Consistency of One-Hot Encoding**: Verify that OneHotEncoder has consistent feature names across folds. Inconsistent feature names could indicate that categories were seen in some folds but not others, potentially leading to data leakage. You can check the categories for each fold:

In [84]:
for idx, estimator in enumerate(cv_results['estimator']):
    print(f"Estimator {idx+1}")
    ohe = estimator.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
    print("Categories for OneHotEncoder:", ohe.categories_)

Estimator 1
Categories for OneHotEncoder: [array(['east', 'north', 'south', 'west'], dtype=object), array(['Ciutat Vella', 'Eixample', 'Gràcia', 'Horta', 'Les Cors',
       'Nou Barris', 'Sant Andreu', 'Sant Martí', 'Sants',
       'Sarrià-Sant Gervasi'], dtype=object), array(['1', '10', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)]
Estimator 2
Categories for OneHotEncoder: [array(['east', 'north', 'south', 'west'], dtype=object), array(['Ciutat Vella', 'Eixample', 'Gràcia', 'Horta', 'Les Cors',
       'Nou Barris', 'Sant Andreu', 'Sant Martí', 'Sants',
       'Sarrià-Sant Gervasi'], dtype=object), array(['1', '10', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)]
Estimator 3
Categories for OneHotEncoder: [array(['east', 'north', 'south', 'west'], dtype=object), array(['Ciutat Vella', 'Eixample', 'Gràcia', 'Horta', 'Les Cors',
       'Nou Barris', 'Sant Andreu', 'Sant Martí', 'Sants',
       'Sarrià-Sant Gervasi'], dtype=object), array(['1', '10', '2', '3', '4', '5', '

It looks like the categories output from `OneHotEncoder` includes the full list rather than showing that it has dropped the first category, despite `drop='first'` being set. This happens because `OneHotEncoder` will still display the full list of categories in `categories_` even if the first category is dropped during encoding. The `categories_` attribute lists all categories found in each feature, not the columns that will actually appear in the transformed data.

To verify that `OneHotEncoder` is correctly dropping the first dummy in each category, you can inspect the transformed output shape or examine the feature names after transformation. Here’s how you can check:

After transforming a sample dataset, retrieve the feature names to confirm which dummies are actually included:

This should confirm that the first category in each feature is indeed being dropped in the actual encoded output.

In [85]:
for idx, estimator in enumerate(cv_results['estimator']):
    ohe_feature_names = estimator.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_cols_v3)
    print(f"Estimator {idx+1} OneHotEncoder feature names: {ohe_feature_names}")

Estimator 1 OneHotEncoder feature names: ['orientation_north' 'orientation_south' 'orientation_west'
 'neighborhood_Eixample' 'neighborhood_Gràcia' 'neighborhood_Horta'
 'neighborhood_Les Cors' 'neighborhood_Nou Barris'
 'neighborhood_Sant Andreu' 'neighborhood_Sant Martí' 'neighborhood_Sants'
 'neighborhood_Sarrià-Sant Gervasi' 'floor_10' 'floor_2' 'floor_3'
 'floor_4' 'floor_5' 'floor_6' 'floor_7' 'floor_8' 'floor_9']
Estimator 2 OneHotEncoder feature names: ['orientation_north' 'orientation_south' 'orientation_west'
 'neighborhood_Eixample' 'neighborhood_Gràcia' 'neighborhood_Horta'
 'neighborhood_Les Cors' 'neighborhood_Nou Barris'
 'neighborhood_Sant Andreu' 'neighborhood_Sant Martí' 'neighborhood_Sants'
 'neighborhood_Sarrià-Sant Gervasi' 'floor_10' 'floor_2' 'floor_3'
 'floor_4' 'floor_5' 'floor_6' 'floor_7' 'floor_8' 'floor_9']
Estimator 3 OneHotEncoder feature names: ['orientation_north' 'orientation_south' 'orientation_west'
 'neighborhood_Eixample' 'neighborhood_Gràcia' 'nei

Therefore, the OHE is working as expected!

---

3. **Confirm Imputation Consistency for Boolean Columns**: Ensure that `SimpleImputer` for boolean columns applies correctly in each fold by comparing the statistics it computed:

In [86]:
for idx, estimator in enumerate(cv_results['estimator']):
    print(f"Estimator {idx+1}")
    boolean_imputer = estimator.named_steps['preprocessor'].named_transformers_['bool']
    print("Most frequent values for Boolean Imputer:", boolean_imputer.statistics_)

Estimator 1
Most frequent values for Boolean Imputer: [False True True True]
Estimator 2
Most frequent values for Boolean Imputer: [False True False True]
Estimator 3
Most frequent values for Boolean Imputer: [False True True True]
Estimator 4
Most frequent values for Boolean Imputer: [False True True True]
Estimator 5
Most frequent values for Boolean Imputer: [False True True True]


Yes, this output suggests that the `SimpleImputer` for boolean columns is working as expected. Each fold has a slightly different set of "most frequent" values across the boolean columns, which means:

1. **Fold-Specific Computation**: The most frequent values vary slightly across folds, which implies that SimpleImputer is independently calculating the mode for each fold's training set rather than using the entire dataset, avoiding data leakage.

2. **Consistent Imputation**: Although you see some consistent values (`True` or `False` in most cases), the occasional change (like `False` in Estimator 2 vs. `True` in others) indicates that imputation is adjusted according to each training subset.

Overall, this output confirms that boolean columns are being imputed correctly, using values calculated independently per fold, ensuring no leakage from the validation set.

#### D. Designing a dynamic way to define the number of groups created by categorical variables for `GroupLasso`

In [28]:
# Checking that, when passing in a certain order the names of the features to a
# data frame, it will return the data frame with the features in that order
df_v3[numerical_cols_v3 + boolean_cols_v3 + categorical_cols_v3]

,num_rooms,num_baths,square_meters,year_built,num_crimes,is_furnished,has_pool,has_ac,accepts_pets,orientation,neighborhood,floor
0,1.0,1.0,NaN,1956.0,2.0,False,False,True,True,NaN,Sant Martí,3
1,4.0,1.0,133.0,1960.0,4.0,False,False,False,False,west,Sants,7
2,2.0,3.0,137.0,2000.0,0.0,False,True,False,False,NaN,Eixample,1
3,1.0,2.0,41.0,2002.0,5.0,False,True,False,False,NaN,Sants,6
4,2.0,1.0,70.0,1979.0,0.0,True,False,False,True,NaN,Gràcia,5
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,2.0,NaN,95.0,2021.0,0.0,False,False,False,False,south,Sant Martí,9
7996,4.0,3.0,166.0,1995.0,9.0,False,False,False,False,NaN,Nou Barris,9
7997,4.0,1.0,89.0,2015.0,0.0,True,False,True,False,east,Sant Martí,9
7998,4.0,2.0,167.0,1977.0,0.0,False,True,True,False,west,Gràcia,8


Since the idea is to define a list for `GroupLasso` which specifies which variables are of the same group, which looks like:

In [73]:
groups_feat_incomp_flat = [
    1,  # 'num_rooms'
    2,  # 'num_baths'
    3,  # 'square_meters'
    4,  # 'year_built'
    5,  # 'num_crimes'
    6,  # 'is_furnished'
    7,  # 'has_pool'
    8,  # 'has_ac'
    9,  # 'accepts_pets'
    
    # Orientation group
    10,  # 'orientation_north'
    10,  # 'orientation_south'
    10,  # 'orientation_west'
    
    # Neighborhood group
    11,  # 'neighborhood_Eixample'
    11,  # 'neighborhood_Gràcia'
    11,  # 'neighborhood_Horta'
    11,  # 'neighborhood_Les Cors'
    11,  # 'neighborhood_Nou Barris'
    11,  # 'neighborhood_Sant Andreu'
    11,  # 'neighborhood_Sant Martí'
    11,  # 'neighborhood_Sants'
    11,  # 'neighborhood_Sarrià-Sant Gervasi'

    # Floor group
    12,  # 'floor_2'
    12,  # 'floor_3'
    12,  # 'floor_4'
    12,  # 'floor_5'
    12,  # 'floor_6'
    12,  # 'floor_7'
    12,  # 'floor_8'
    12,  # 'floor_9'
    12,  # 'floor_10'
]

Then, by passing to `cross-validate` (which is where `GroupLasso` is applied, *after* preprocessing) a dataframe with the order: 1) Numerical columns, 2) Boolean (dummy) columns and 3) Categorical columns, the groups can be defined as 1 number, starting from 1, to numerical columns and boolean columns, and then 1 number per categorical variable which repeats n-1 times, where n = number of unique categories, excluding null values. 

In [33]:
groups_numerical_vars = [x for x in range(1, len(numerical_cols_v3) + 1)] # We add 1 to the end of the range as range() stops before the last number
groups_numerical_vars

[1, 2, 3, 4, 5]

In [35]:
groups_bool_vars = [x for x in range(1, len(boolean_cols_v3) + 1)] # We add 1 to the end of the range as range() stops before the last number
groups_bool_vars

[1, 2, 3, 4]

In [39]:
df_v3['orientation'].nunique(dropna = True) # Count does not include NaN

4

Then, we can create the groups for categorical variables as follows (note how below we don't add 1 since we will have n-1 dummies):

In [44]:
groups_categorical_vars = [1 for x in range(1, df_v3[categorical_cols_v3[0]].nunique(dropna = True))] + [2 for x in range(1, df_v3[categorical_cols_v3[1]].nunique(dropna = True))]
groups_categorical_vars

[1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2]

To increase readability, we can convert this into a loop:

In [50]:
groups_categorical_vars = []
# For every categorical variable in the list (i)
for i in range(0, len(categorical_cols_v3)):
    # For every number of unique categories (-1) in the categorical variable
    for j in range(1, df_v3[categorical_cols_v3[i]].nunique(dropna = True)):
        groups_categorical_vars.append(i + 1)

groups_categorical_vars

[1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Finally, we should define the group so that one list builds on top of the other, with numerical variables starting the count, followed by booleans and then by categorical variables:

In [78]:
# First, we initialize the list with the groups for numerical variables
groups = [x for x in range(1, len(numerical_cols_v3) + 1)]

# Then, we add to the groups list the boolean variables
groups = groups + [x + len(groups) for x in range(1, len(boolean_cols_v3) + 1)]

# Finally, we add to the groups list the categorical variables
# We save the initial length of the groups list in each iteration here, so that 
# the length does not change when we add one more for each category below
start_cat_index = len(groups) + 1
for i, col in enumerate(categorical_cols_v3):
    unique_count = df_v3[col].nunique(dropna=True)  # Get the number of unique categories
    groups.extend([start_cat_index + i] * (unique_count - 1))  # Assign the same group ID to each unique category

groups

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12]

This works as expected!

#### E. Checking that the preprocessor pipeline works together with GroupLasso

In [90]:
# Create transformers for categorical and numerical features
numerical_transformer = [
    ('scaler', StandardScaler())  # Standardize numerical features
    ]
    
categorical_transformer = [
    ('onehot', OneHotEncoder(handle_unknown='error', drop = 'first'))  # One-hot encode categorical features, dropping first dummy
]

my_n_neighbors = 5

# Add KNN imputer for numerical features. We append it, so that standardization is
# performed before KNN Imputing
numerical_transformer.append(('knn_imputer', KNNImputer(n_neighbors = my_n_neighbors)))
# Optionally, use SimpleImputer for categorical features instead of KNN. We insert it 
# BEFORE one-hot encoding the categorical variables
categorical_transformer.insert(0, ('simple_imputer', SimpleImputer(strategy='most_frequent')))

# Create the ColumnTransformer to apply different transformations to numerical
# columns, categorical and dummy (boolean) columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(numerical_transformer), numerical_cols_v3),
        ('cat', Pipeline(categorical_transformer), categorical_cols_v3),
        ('bool', SimpleImputer(strategy='most_frequent'), boolean_cols_v3)
    ]
)

# Create steps of the pipeline
steps = [('preprocessor', preprocessor)]

###############################################################################

# Creating the groups for GroupLasso

# First, we initialize the list with the groups for numerical variables
lasso_groups = [x for x in range(1, len(numerical_cols_v3) + 1)]

# Then, we add to the groups list the boolean variables
lasso_groups = lasso_groups + [x + len(lasso_groups) for x in range(1, len(boolean_cols_v3) + 1)]

# Finally, we add to the groups list the categorical variables
# We save the initial length of the groups list in each iteration here, so that 
# the length does not change when we add one more for each category below
start_cat_index = len(lasso_groups) + 1
for i, col in enumerate(categorical_cols_v3):
    unique_count = df_v3[col].nunique(dropna=True)  # Get the number of unique categories
    lasso_groups.extend([start_cat_index + i] * (unique_count - 1))  # Assign the same group ID to each unique category

###############################################################################

steps.append(('group_lasso', 
                        GroupLasso(
                            groups = lasso_groups, 
                            group_reg = 0.05, 
                            l1_reg = 0.05, 
                            fit_intercept = True, 
                            n_iter = 500,
                            scale_reg = "group_size",
                            supress_warning = True
                            )))
            
# We create the pipeline with the steps specified above
pipeline = Pipeline(steps)

###############################################################################

# We perform cross-validation

target = 'price'
n_folds = 5

cv_results = cross_validate(
                estimator = pipeline, 
                X = df_v3[features_v3], y = df_v3[target], cv = n_folds, 
                scoring='neg_mean_squared_error',
                return_estimator=True
                )

***Methods to Validate Code Execution and Results***

To ensure the code performs as expected, try the following:

1. **Verify Preprocessing Steps**  
   - **Inspect Dummy Variables**: After cross-validation, check the structure of dummy variables to confirm that the first category has been consistently dropped. You can retrieve the first estimator’s pipeline and output transformed columns, as in: 

In [87]:
# Retrieve one pipeline for inspection
first_estimator = cv_results['estimator'][0]
transformed_data = first_estimator.named_steps['preprocessor'].transform(df_v3[features_v3])

# Check the columns for expected transformations
categorical_transformed_names = first_estimator.named_steps['preprocessor'] \
                                .named_transformers_['cat'] \
                                .named_steps['onehot'] \
                                .get_feature_names_out(categorical_cols_v3)

print("Transformed categorical features:", categorical_transformed_names)

Transformed categorical features: ['orientation_north' 'orientation_south' 'orientation_west'
 'neighborhood_Eixample' 'neighborhood_Gràcia' 'neighborhood_Horta'
 'neighborhood_Les Cors' 'neighborhood_Nou Barris'
 'neighborhood_Sant Andreu' 'neighborhood_Sant Martí' 'neighborhood_Sants'
 'neighborhood_Sarrià-Sant Gervasi' 'floor_10' 'floor_2' 'floor_3'
 'floor_4' 'floor_5' 'floor_6' 'floor_7' 'floor_8' 'floor_9']


The first category has been dropped!

2. **Validate Grouping for Group Lasso**
   - Print `lasso_groups` to verify that it matches the expected format and group counts for numerical, boolean, and categorical columns:
   - Ensure that the length of `lasso_groups` aligns with the total number of columns post-transformation.

In [88]:
print("Lasso Groups:", lasso_groups)

Lasso Groups: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12]


Groups match the length of columns post-transformation!

3. **Cross-Validation Results Check**
   - Inspect the `cv_results` to check both the mean cross-validated score and individual fold scores:
   - If possible, validate the shape and types of `cv_results['estimator']` to confirm that each step executed without error across all folds.

In [91]:
# Mean and std deviation of cross-validated negative MSE scores
mean_score = np.mean(cv_results['test_score'])
std_score = np.std(cv_results['test_score'])
print(f"Mean CV score: {mean_score}, Std Dev of CV score: {std_score}")

Mean CV score: -31067.83582374016, Std Dev of CV score: 937.7856589639939


4. **Final Validation by Output Inspection**  
   - Check the `transformed_data` shape matches the expected column count (number of numerical + number of one-hot encoded + number of boolean).
   - Optionally, apply `pipeline.fit(df_v3[features_v3], df_v3[target])` to the entire dataset and compare predictions or transformed features against known results or expectations, ensuring the pipeline works outside of cross-validation.

These checks should give you confidence that the pipeline and cross-validation workflow operate as expected and handle preprocessing and grouping correctly across all stages.

Everything seems good to go!! Now, everything can be joined in the function.

### Definition of the function

In [15]:
def group_lasso_imputers(
        df: pd.DataFrame, numerical_features: list[str], boolean_features: list[str], 
        categorical_features: list[str],target: str, n_folds: int = 5, 
        use_imputers: bool = False, my_n_neighbors: int = 5, max_n_lasso_iter: int = 100
        ):

    ###########################################################################

    results_list = []  # To store all results we want for our data frame
    alpha_counter = 0  # Initialize a counter for the alphas
    group_reg_parameter_counter = 0 # Initialize a counter for the group regularization parameter
    features = numerical_features + boolean_features + categorical_features

    ###########################################################################

    # Create transformers for categorical and numerical features
    numerical_transformer = [('scaler', StandardScaler())]  # Standardize numerical features
        
    categorical_transformer = [('onehot', OneHotEncoder(handle_unknown='error', drop = 'first'))]  # One-hot encode categorical features, dropping first dummy

    if use_imputers == True:
        # Add KNN imputer for numerical features. We append it, so that standardization is
        # performed before KNN Imputing
        numerical_transformer.append(('knn_imputer', KNNImputer(n_neighbors = my_n_neighbors)))
        # Optionally, use SimpleImputer for categorical features instead of KNN. We insert it 
        # BEFORE one-hot encoding the categorical variables
        categorical_transformer.insert(0, ('simple_imputer', SimpleImputer(strategy='most_frequent')))

    # Set the boolean transformer based on use_imputers
    if use_imputers == True:
        # Optionally, use SimpleImputer for boolean features
        boolean_transformer = [('simple_imputer', SimpleImputer(strategy='most_frequent'))]
    else:
        # Pass through boolean features without transformation
        boolean_transformer = [('passthrough', 'passthrough')]

    # Create the ColumnTransformer to apply different transformations to numerical
    # columns, categorical and dummy (boolean) columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline(numerical_transformer), numerical_features),
            ('cat', Pipeline(categorical_transformer), categorical_features),
            ('bool', Pipeline(boolean_transformer), boolean_features)
        ]
    )

    # Create steps of the pipeline, which begins with the preprocessor. It will
    # apply the transformation and imputations (if use_imputers is set to True) 
    # to the different features in the order specified in ColumnTransformer
    steps = [('preprocessor', preprocessor)]
    
    ###########################################################################

    # We create a list with the definition of the groups for GroupLasso

    # First, we initialize the list with the groups for numerical variables
    lasso_groups = [x for x in range(1, len(numerical_features) + 1)]

    # Then, we add to the groups list the boolean variables
    lasso_groups = lasso_groups + [x + len(lasso_groups) for x in range(1, len(boolean_features) + 1)]

    # Finally, we add to the groups list the categorical variables
    # We save the initial length of the groups list in each iteration here, so that 
    # the length does not change when we add one more for each category below
    start_cat_index = len(lasso_groups) + 1
    for i, col in enumerate(categorical_features):
        unique_count = df[col].nunique(dropna=True)  # Get the number of unique categories, ignoring NaN
        lasso_groups.extend([start_cat_index + i] * (unique_count - 1))  # Assign the same group ID to each unique category 

    ###########################################################################

    # Now, we create a first loop that iterates over different values of the alpha
    # parameter. We generate 50 alphas
    for alpha in np.logspace(-10, 0, 50):
    
        # We add 1 to the counter of alpha each time we consider another alpha
        alpha_counter += 1

        # We reset the counter of the group_parameter after the nested loop ends
        group_reg_parameter_counter = 0

        # We create a second loop that iterates over different values of the 
        # parameter that penalizes group sparsity
        for group_parameter in np.logspace(-10, 0, 50):
            
            # We add 1 to the counter of the group regularization parameter in each iteration
            group_reg_parameter_counter += 1

            # We add Group Lasso to the pipeline, with a specific alpha
            steps.append(('group_lasso', 
                        GroupLasso(
                            groups = lasso_groups, 
                            group_reg = group_parameter, 
                            l1_reg = alpha, 
                            fit_intercept = True, 
                            n_iter = max_n_lasso_iter,
                            scale_reg = "group_size",
                            supress_warning = True
                            )))
            
            # We create the pipeline with the steps specified above
            pipeline = Pipeline(steps)

            # Perform cross-validation: on each fold, it will scale and fit the training
            # data, and compute the negative mean squared error (so a higher value, i.e.,
            # more negative, is better).

            cv_results = cross_validate(
                estimator = pipeline, 
                X = df[features], 
                y = df[target], cv = n_folds, scoring = 'neg_mean_squared_error',
                return_estimator = True
                )

            neg_mse = cv_results['test_score']  # Negative MSE scores
            estimators = cv_results['estimator']  # List of fitted estimators for each fold
            
            # Loop through each fold result to store the information
            for index, (score, estimator) in enumerate(zip(neg_mse, estimators)):
                # We access the information associated to the group lasso
                group_lasso = estimator.named_steps['group_lasso']
                # We store the coefficients estimated by group lasso
                coefs = group_lasso.coef_
                # We store the intercept estimated by group lasso
                intercept = group_lasso.intercept_[0]

                # Create a dictionary to store the results for this fold
                result_dict = {
                    'Fold': index + 1,
                    'Alpha_counter': alpha_counter,
                    'Group_reg_parameter_counter': group_reg_parameter_counter,
                    'Alpha': alpha,
                    'Group_reg_parameter': group_parameter,
                    'Negative MSE': score,
                    'Intercept': intercept
                }

                # Get feature names after one-hot encoding
                cat_feature_names = estimator.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
                combined_features = np.concatenate([numerical_features, boolean_features, cat_feature_names])

                # Assign coefficients to the corresponding features
                for i, feature in enumerate(combined_features):
                    result_dict[feature] = coefs[i][0]

                # Append the result to the list
                results_list.append(result_dict)
    
            # Lastly, we remove the last step of the pipeline so that the 
            # GroupLasso step does not accumulate with different hyperparameters.
            # We do this for each iteration of group_reg
            steps.pop()

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results_list)

    return results_df

### Training with function 4

#### Test

In [11]:
# Test with function 4
# results_v3_f4_test = group_lasso_imputers(df = df_v3, numerical_features = ['square_meters'], boolean_features = ['has_ac'], categorical_features = ['orientation'], target = 'price', n_folds = 5, use_imputers = True, my_n_neighbors = 5, max_n_lasso_iter = 500)

# results_v3_f4_test.to_csv('3_results_datasets/results_v3_f4_test.csv')

This test has lasted for 44 minutes (with only one numerical, one dummy and one categorical variable!). Changes to apply: reset counter `group_reg_parameter_counter` after all of the combinations have been tried (maybe define it as equal to 0 in the first loop).

In [ ]:
# results_v3_f4_test

#### Training with all of the features

In [16]:
# results_v3_f4 = group_lasso_imputers(df = df_v3, numerical_features = numerical_cols_v3, boolean_features = boolean_cols_v3, categorical_features = categorical_cols_v3, target = 'price', n_folds = 5, use_imputers = True, my_n_neighbors = 5, max_n_lasso_iter = 500)
# results_v3_f4.to_csv('3_results_datasets/results_v3_f4.csv')

/home/pablo/miniconda3/envs/py312/lib/python3.12/site-packages/group_lasso/_fista.py:114: ConvergenceWarning: The FISTA iterations did not converge to a sufficient minimum.
You used subsampling then this is expected, otherwise, try increasing the number of iterations or decreasing the tolerance.
  warnings.warn(
/home/pablo/miniconda3/envs/py312/lib/python3.12/site-packages/group_lasso/_fista.py:114: ConvergenceWarning: The FISTA iterations did not converge to a sufficient minimum.
You used subsampling then this is expected, otherwise, try increasing the number of iterations or decreasing the tolerance.
  warnings.warn(
/home/pablo/miniconda3/envs/py312/lib/python3.12/site-packages/group_lasso/_fista.py:114: ConvergenceWarning: The FISTA iterations did not converge to a sufficient minimum.
You used subsampling then this is expected, otherwise, try increasing the number of iterations or decreasing the tolerance.
  warnings.warn(
/home/pablo/miniconda3/envs/py312/lib/python3.12/site-pack

It has been 160 minutes doing the computations, and in approx. 15 folds it did not converge.

In [ ]:
# results_v3_f4

## Function 5: function 4 but with regular linear regression instead and trying all possible combinations (treating dummies as a unit)
The idea here would be to impute null values in the same way as in function 4, but instead try all possible combinations of linear regression as implemented in function 1. This function, `lin_reg_with_preprocessing_combos`, performs linear regression with cross-validation on all possible combinations of features (numerical, boolean, and categorical) in a dataset, while dynamically applying preprocessing steps based on the types of features present in each combination. Key steps:

1. **Feature Combinations:** It generates all possible subsets of features (combinations of numerical, boolean, and categorical) to test various models with different feature groups.
  
2. **Dynamic Preprocessing:** 
   - For each combination, it builds a `ColumnTransformer` that includes preprocessing steps only for the types of features present:
     - **Numerical Features**: Standard scaling and optional KNN imputation.
     - **Categorical Features**: One-hot encoding (dropping the first category) and optional most-frequent imputation.
     - **Boolean Features**: Optional most-frequent imputation or passthrough.
     
3. **Pipeline and Cross-Validation:** 
   - For each feature subset, it creates a pipeline with the `ColumnTransformer` and a linear regression model.
   - Cross-validation is performed on the pipeline, returning the negative mean squared error (MSE) and fitted model for each fold.

4. **Result Storage and Output:**
   - For each fold and combination, it records:
     - Model performance (negative MSE).
     - Model coefficients and intercept.
     - Feature coefficients are mapped to feature names, and features not in the combination are assigned `NaN`.
   - Results for all combinations and folds are stored in a DataFrame and returned.

In [13]:
def lin_reg_with_preprocessing_combos(
    df: pd.DataFrame, numerical_features: list[str], boolean_features: list[str], 
    categorical_features: list[str], target: str, n_folds: int = 5, use_imputers: bool = False,
    my_n_neighbors: int = 5
):

    def flatten_combo(combo):
        """
        Flattens the combination by including all variables from nested lists
        when one element of a list is selected.
        """
        return list(itertools.chain(*[x if isinstance(x, list) else [x] for x in combo]))

    results_list = []  # To store all results we want for our data frame
    combo_counter = 0  # Initialize a counter for the combinations
    features = numerical_features + boolean_features + categorical_features

    for r in range(1, len(features) + 1):
        for combo in itertools.combinations(features, r):

            combo_counter += 1

            # We flatten the list of the resulting combination of features
            flat_combo = flatten_combo(combo)

            # Determine which types of features are present in the current combination
            current_numerical_features = [f for f in flat_combo if f in numerical_features]
            current_boolean_features = [f for f in flat_combo if f in boolean_features]
            current_categorical_features = [f for f in flat_combo if f in categorical_features]

            # Build the transformers dynamically based on the present feature types
            transformers = []
            
            # Add numerical transformer if numerical features are present
            if current_numerical_features:
                numerical_transformer = [('scaler', StandardScaler())]
                if use_imputers:
                    numerical_transformer.append(('knn_imputer', KNNImputer(n_neighbors=my_n_neighbors)))
                transformers.append(('num', Pipeline(numerical_transformer), current_numerical_features))
                
            # Add categorical transformer if categorical features are present
            if current_categorical_features:
                categorical_transformer = [('onehot', OneHotEncoder(handle_unknown='error', drop='first'))]
                if use_imputers:
                    categorical_transformer.insert(0, ('simple_imputer', SimpleImputer(strategy='most_frequent')))
                transformers.append(('cat', Pipeline(categorical_transformer), current_categorical_features))
                
            # Add boolean transformer if boolean features are present
            if current_boolean_features:
                if use_imputers:
                    boolean_transformer = [('simple_imputer', SimpleImputer(strategy='most_frequent'))]
                else:
                    boolean_transformer = [('passthrough', 'passthrough')]
                transformers.append(('bool', Pipeline(boolean_transformer), current_boolean_features))
                
            # Define the preprocessor with only the applicable transformers
            preprocessor = ColumnTransformer(transformers=transformers)

            # Pipeline: apply transformations and linear regression
            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('linreg', LinearRegression(fit_intercept=True))
            ])

            # Perform cross-validation and get negative MSE scores
            cv_results = cross_validate(
                estimator=pipeline,
                X=df[flat_combo],
                y=df[target],
                cv=n_folds,
                scoring='neg_mean_squared_error',
                return_estimator=True
            )

            neg_mse = cv_results['test_score']
            estimators = cv_results['estimator']

            # Store results for each fold
            for index, (score, estimator) in enumerate(zip(neg_mse, estimators)):
                linreg = estimator.named_steps['linreg']
                coefs = linreg.coef_
                intercept = linreg.intercept_

                result_dict = {
                    'Fold': index + 1,
                    'Combination ID': combo_counter,
                    'Negative MSE': score,
                    'Intercept': intercept
                }

                # Get feature names post one-hot encoding and exclude original categorical columns
                if current_categorical_features:
                    cat_feature_names = estimator.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(current_categorical_features)
                else:
                    cat_feature_names = []

                # Combine the feature names, excluding the original categorical columns
                combined_features = np.concatenate([current_numerical_features, current_boolean_features, cat_feature_names])

                # Assign coefficients to the corresponding features
                for i, feature in enumerate(combined_features):
                    result_dict[feature] = coefs[i]

                # Assign NaN for features not in the current combination, excluding original categorical columns
                for feature in features:
                    if feature in numerical_features or feature in boolean_features:
                        if feature not in combined_features:
                            result_dict[feature] = float('nan')

                results_list.append(result_dict)

    # Convert results to DataFrame
    results_df = pd.DataFrame(results_list)
    return results_df

### Training with function 5
Now, we apply the function to dataset 3.

In [15]:
# results_v3_f5 = lin_reg_with_preprocessing_combos(
    # df = df_v3, numerical_features = numerical_cols_v3, boolean_features = boolean_cols_v3,
    # categorical_features = categorical_cols_v3, target = 'price', n_folds = 5,
    # use_imputers = True, my_n_neighbors = 5
# )

# results_v3_f5.to_csv('3_results_datasets/results_v3_f5.csv')

In [12]:
# results_v3_f5

,Fold,Combination ID,Negative MSE,Intercept,num_rooms,num_baths,square_meters,year_built,num_crimes,is_furnished,...,neighborhood_Sarrià-Sant Gervasi,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9
0,1,1,-72989.051984,1099.686250,5.927694,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,-70145.790556,1101.458125,11.108159,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,-75719.267630,1097.982500,7.062621,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,-74018.007181,1099.884531,9.092078,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,-75968.024362,1099.069844,8.212737,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20470,1,4095,-32586.625413,1058.615650,3.917668,7.126842,192.101034,-0.757726,-75.904084,-8.141981,...,33.428130,44.272879,56.444276,57.885396,73.404529,76.642061,2.013203,3.776742,10.158322,-2.401222
20471,2,4095,-31189.621399,1053.414410,4.476038,7.996682,193.742519,0.812930,-75.533129,-2.849968,...,35.992292,36.003038,52.651858,66.409417,75.126816,74.616775,4.049733,6.425065,4.969810,-1.908622
20472,3,4095,-29805.370984,1047.106412,4.916282,6.317971,188.796832,-1.523096,-76.388400,-1.363997,...,37.926515,45.501094,61.977180,66.666239,69.623761,78.723152,1.894611,6.227323,5.901172,-0.808511
20473,4,4095,-30412.817955,1044.818466,4.711976,9.492917,189.692961,-0.766892,-76.860957,1.470256,...,26.221743,38.361906,57.378565,60.068034,70.781013,75.278541,6.783332,5.619868,8.575298,2.863775


## Function 6: function 4 but with regular Lasso instead

In [47]:
def lasso_imputers(
        df: pd.DataFrame, numerical_features: list[str], boolean_features: list[str], 
        categorical_features: list[str],target: str, n_folds: int = 5, 
        use_imputers: bool = False, my_n_neighbors: int = 5, max_n_lasso_iter: int = 100
        ):

    ###########################################################################

    results_list = []  # To store all results we want for our data frame
    alpha_counter = 0  # Initialize a counter for the alphas
    features = numerical_features + boolean_features + categorical_features

    ###########################################################################

    # Create transformers for categorical and numerical features
    numerical_transformer = [('scaler', StandardScaler())]  # Standardize numerical features
        
    categorical_transformer = [('onehot', OneHotEncoder(handle_unknown='error', drop = 'first'))]  # One-hot encode categorical features, dropping first dummy

    if use_imputers == True:
        # Add KNN imputer for numerical features. We append it, so that standardization is
        # performed before KNN Imputing
        numerical_transformer.append(('knn_imputer', KNNImputer(n_neighbors = my_n_neighbors)))
        # Optionally, use SimpleImputer for categorical features instead of KNN. We insert it 
        # BEFORE one-hot encoding the categorical variables
        categorical_transformer.insert(0, ('simple_imputer', SimpleImputer(strategy='most_frequent')))

    # Set the boolean transformer based on use_imputers
    if use_imputers == True:
        # Optionally, use SimpleImputer for boolean features
        boolean_transformer = [('simple_imputer', SimpleImputer(strategy='most_frequent'))]
    else:
        # Pass through boolean features without transformation
        boolean_transformer = [('passthrough', 'passthrough')]

    # Create the ColumnTransformer to apply different transformations to numerical
    # columns, categorical and dummy (boolean) columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline(numerical_transformer), numerical_features),
            ('cat', Pipeline(categorical_transformer), categorical_features),
            ('bool', Pipeline(boolean_transformer), boolean_features)
        ]
    )

    # Create steps of the pipeline, which begins with the preprocessor. It will
    # apply the transformation and imputations (if use_imputers is set to True) 
    # to the different features in the order specified in ColumnTransformer
    steps = [('preprocessor', preprocessor)]
    
    ###########################################################################

    # Now, we create a loop that iterates over different values of the alpha
    # parameter. We generate 50 alphas
    for alpha in np.logspace(-10, 0, 50):
    
        # We add 1 to the counter of alpha each time we consider another alpha
        alpha_counter += 1

        # We add Lasso to the pipeline, with a specific alpha
        steps.append(('lasso', 
                    Lasso(alpha = alpha,
                          fit_intercept = True,
                          max_iter = max_n_lasso_iter
                        )))
        
        # We create the pipeline with the steps specified above
        pipeline = Pipeline(steps)

        # Perform cross-validation: on each fold, it will scale and fit the training
        # data, and compute the negative mean squared error (so a higher value, i.e.,
        # more negative, is better).

        cv_results = cross_validate(
            estimator = pipeline, 
            X = df[features], 
            y = df[target], cv = n_folds, scoring = 'neg_mean_squared_error',
            return_estimator = True
            )

        neg_mse = cv_results['test_score']  # Negative MSE scores
        estimators = cv_results['estimator']  # List of fitted estimators for each fold
        
        # Loop through each fold result to store the information
        for index, (score, estimator) in enumerate(zip(neg_mse, estimators)):
            # We access the information associated to the lasso
            lasso = estimator.named_steps['lasso']
            # We store the coefficients estimated by lasso
            coefs = lasso.coef_
            # We store the intercept estimated by lasso
            intercept = lasso.intercept_

            # Create a dictionary to store the results for this fold
            result_dict = {
                'Fold': index + 1,
                'Alpha_counter': alpha_counter,
                'Alpha': alpha,
                'Negative MSE': score,
                'Intercept': intercept
            }

            # Get feature names after one-hot encoding
            cat_feature_names = estimator.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
            combined_features = np.concatenate([numerical_features, boolean_features, cat_feature_names])

            # Assign coefficients to the corresponding features
            for i, feature in enumerate(combined_features):
                result_dict[feature] = coefs[i]

            # Append the result to the list
            results_list.append(result_dict)

        # Lastly, we remove the last step of the pipeline so that the 
        # Lasso step does not accumulate with different hyperparameters.
        # We do this for each iteration of alpha
        steps.pop()

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results_list)

    return results_df

### Training with function 6

In [49]:
# results_v3_f6 = lasso_imputers(
    # df = df_v3, numerical_features = numerical_cols_v3, boolean_features = boolean_cols_v3, 
    # categorical_features = categorical_cols_v3, target = 'price', n_folds = 5, use_imputers = True, my_n_neighbors = 5, 
    # max_n_lasso_iter = 500)

# results_v3_f6.to_csv('3_results_datasets/results_v3_f6.csv')

In [25]:
# results_v3_f6

,Fold,Alpha_counter,Alpha,Negative MSE,Intercept,num_rooms,num_baths,square_meters,year_built,num_crimes,...,neighborhood_Sarrià-Sant Gervasi,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9
0,1,1,1.000000e-10,-32586.625413,1058.615650,3.917668,7.126842,192.101034,-0.757726,-75.904084,...,33.428130,44.272879,56.444276,57.885396,73.404529,76.642061,2.013203,3.776742,10.158322,-2.401222
1,2,1,1.000000e-10,-31189.621399,1053.414410,4.476038,7.996682,193.742519,0.812930,-75.533129,...,35.992292,36.003038,52.651858,66.409417,75.126816,74.616775,4.049733,6.425065,4.969810,-1.908622
2,3,1,1.000000e-10,-29805.370984,1047.106412,4.916282,6.317971,188.796832,-1.523096,-76.388400,...,37.926515,45.501094,61.977180,66.666239,69.623761,78.723152,1.894611,6.227323,5.901172,-0.808511
3,4,1,1.000000e-10,-30412.817955,1044.818466,4.711976,9.492917,189.692961,-0.766892,-76.860957,...,26.221743,38.361906,57.378565,60.068034,70.781013,75.278541,6.783332,5.619868,8.575298,2.863775
4,5,1,1.000000e-10,-31405.396078,1049.849450,3.797850,5.971341,189.547885,-0.095415,-75.176167,...,24.830778,33.015063,49.250481,64.616720,62.177033,71.770241,2.165158,7.505489,8.875008,0.785281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1,50,1.000000e+00,-32604.351129,1088.581253,2.884792,6.277713,191.155709,-0.000000,-75.716975,...,-0.000000,0.000000,9.460519,10.103815,25.372687,30.161026,0.000000,0.000000,5.730685,-0.000000
246,2,50,1.000000e+00,-31242.094652,1089.915224,3.381021,7.204187,192.503337,0.000000,-75.412434,...,0.000000,0.000000,9.331053,21.120634,29.966812,30.335900,0.000000,2.202022,0.801199,-0.000000
247,3,50,1.000000e+00,-29881.521004,1089.562456,3.758123,5.507426,187.578657,-0.381861,-75.710976,...,-0.000000,0.000000,12.680365,17.250843,20.419509,29.721570,0.000000,1.628526,1.271786,-0.000000
248,4,50,1.000000e+00,-30471.671817,1086.952384,3.670738,8.564819,188.503710,-0.000000,-76.605951,...,-0.000000,0.000000,14.390252,16.481862,27.912827,32.294465,1.952138,1.181696,4.143621,0.000000


## Function 7: function 6 but with Ridge

In [27]:
def ridge_imputers(
        df: pd.DataFrame, numerical_features: list[str], boolean_features: list[str], 
        categorical_features: list[str],target: str, n_folds: int = 5, 
        use_imputers: bool = False, my_n_neighbors: int = 5, max_n_ridge_iter: int = 100
        ):

    ###########################################################################

    results_list = []  # To store all results we want for our data frame
    alpha_counter = 0  # Initialize a counter for the alphas
    features = numerical_features + boolean_features + categorical_features

    ###########################################################################

    # Create transformers for categorical and numerical features
    numerical_transformer = [('scaler', StandardScaler())]  # Standardize numerical features
        
    categorical_transformer = [('onehot', OneHotEncoder(handle_unknown='error', drop = 'first'))]  # One-hot encode categorical features, dropping first dummy

    if use_imputers == True:
        # Add KNN imputer for numerical features. We append it, so that standardization is
        # performed before KNN Imputing
        numerical_transformer.append(('knn_imputer', KNNImputer(n_neighbors = my_n_neighbors)))
        # Optionally, use SimpleImputer for categorical features instead of KNN. We insert it 
        # BEFORE one-hot encoding the categorical variables
        categorical_transformer.insert(0, ('simple_imputer', SimpleImputer(strategy='most_frequent')))

    # Set the boolean transformer based on use_imputers
    if use_imputers == True:
        # Optionally, use SimpleImputer for boolean features
        boolean_transformer = [('simple_imputer', SimpleImputer(strategy='most_frequent'))]
    else:
        # Pass through boolean features without transformation
        boolean_transformer = [('passthrough', 'passthrough')]

    # Create the ColumnTransformer to apply different transformations to numerical
    # columns, categorical and dummy (boolean) columns
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline(numerical_transformer), numerical_features),
            ('cat', Pipeline(categorical_transformer), categorical_features),
            ('bool', Pipeline(boolean_transformer), boolean_features)
        ]
    )

    # Create steps of the pipeline, which begins with the preprocessor. It will
    # apply the transformation and imputations (if use_imputers is set to True) 
    # to the different features in the order specified in ColumnTransformer
    steps = [('preprocessor', preprocessor)]
    
    ###########################################################################

    # Now, we create a loop that iterates over different values of the alpha
    # parameter. We generate 50 alphas
    for alpha in np.logspace(-10, 0, 50):
    
        # We add 1 to the counter of alpha each time we consider another alpha
        alpha_counter += 1

        # We add Ridge to the pipeline, with a specific alpha
        steps.append(('ridge', 
                    Ridge(alpha = alpha,
                          fit_intercept = True,
                          max_iter = max_n_ridge_iter
                        )))
        
        # We create the pipeline with the steps specified above
        pipeline = Pipeline(steps)

        # Perform cross-validation: on each fold, it will scale and fit the training
        # data, and compute the negative mean squared error (so a higher value, i.e.,
        # more negative, is better).

        cv_results = cross_validate(
            estimator = pipeline, 
            X = df[features], 
            y = df[target], cv = n_folds, scoring = 'neg_mean_squared_error',
            return_estimator = True
            )

        neg_mse = cv_results['test_score']  # Negative MSE scores
        estimators = cv_results['estimator']  # List of fitted estimators for each fold
        
        # Loop through each fold result to store the information
        for index, (score, estimator) in enumerate(zip(neg_mse, estimators)):
            # We access the information associated to the ridge
            ridge = estimator.named_steps['ridge']
            # We store the coefficients estimated by ridge
            coefs = ridge.coef_
            # We store the intercept estimated by ridge
            intercept = ridge.intercept_

            # Create a dictionary to store the results for this fold
            result_dict = {
                'Fold': index + 1,
                'Alpha_counter': alpha_counter,
                'Alpha': alpha,
                'Negative MSE': score,
                'Intercept': intercept
            }

            # Get feature names after one-hot encoding
            cat_feature_names = estimator.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)
            combined_features = np.concatenate([numerical_features, boolean_features, cat_feature_names])

            # Assign coefficients to the corresponding features
            for i, feature in enumerate(combined_features):
                result_dict[feature] = coefs[i]

            # Append the result to the list
            results_list.append(result_dict)

        # Lastly, we remove the last step of the pipeline so that the 
        # Ridge step does not accumulate with different hyperparameters.
        # We do this for each iteration of alpha
        steps.pop()

    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results_list)

    return results_df

### Training with function 7

In [28]:
# results_v3_f7 = ridge_imputers(
    # df = df_v3, numerical_features = numerical_cols_v3, boolean_features = boolean_cols_v3, 
    # categorical_features = categorical_cols_v3, target = 'price', n_folds = 5, use_imputers = True, my_n_neighbors = 5, 
    # max_n_ridge_iter = 500)

# results_v3_f7.to_csv('3_results_datasets/results_v3_f7.csv')

In [29]:
# results_v3_f7

,Fold,Alpha_counter,Alpha,Negative MSE,Intercept,num_rooms,num_baths,square_meters,year_built,num_crimes,...,neighborhood_Sarrià-Sant Gervasi,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9
0,1,1,1.000000e-10,-32586.625413,1058.615650,3.917668,7.126842,192.101034,-0.757726,-75.904084,...,33.428130,44.272879,56.444276,57.885396,73.404529,76.642061,2.013203,3.776742,10.158322,-2.401222
1,2,1,1.000000e-10,-31189.621399,1053.414410,4.476038,7.996682,193.742519,0.812930,-75.533129,...,35.992292,36.003038,52.651858,66.409417,75.126816,74.616775,4.049733,6.425065,4.969810,-1.908622
2,3,1,1.000000e-10,-29805.370984,1047.106412,4.916282,6.317971,188.796832,-1.523096,-76.388400,...,37.926515,45.501094,61.977180,66.666239,69.623761,78.723152,1.894611,6.227323,5.901172,-0.808511
3,4,1,1.000000e-10,-30412.817955,1044.818466,4.711976,9.492917,189.692961,-0.766892,-76.860957,...,26.221743,38.361906,57.378565,60.068034,70.781013,75.278541,6.783332,5.619868,8.575298,2.863775
4,5,1,1.000000e-10,-31405.396078,1049.849450,3.797850,5.971341,189.547885,-0.095415,-75.176167,...,24.830778,33.015063,49.250481,64.616720,62.177033,71.770241,2.165158,7.505489,8.875008,0.785281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1,50,1.000000e+00,-32584.184696,1059.284122,3.916763,7.128586,192.073894,-0.756566,-75.891041,...,32.634630,43.477891,55.621420,57.057030,72.547391,75.790454,2.007384,3.766623,10.142581,-2.414504
246,2,50,1.000000e+00,-31187.627056,1054.139730,4.476951,7.997505,193.708590,0.814695,-75.521024,...,35.192077,35.203529,51.839256,65.561366,74.259067,73.760643,4.033803,6.417302,4.964618,-1.913205
247,3,50,1.000000e+00,-29804.112576,1047.868965,4.916530,6.319826,188.764847,-1.520350,-76.375991,...,37.112792,44.690726,61.128867,65.811470,68.759782,77.851944,1.885387,6.210743,5.884575,-0.818417
248,4,50,1.000000e+00,-30413.765746,1045.545511,4.711312,9.492476,189.660485,-0.765057,-76.848688,...,25.487581,37.617542,56.597949,59.279474,69.975081,74.468273,6.770129,5.608413,8.560664,2.856791


## Function 8: function 5 but with imputation of median for numerical variables

In [9]:
def lin_reg_with_preprocessing_combos_median_imputing(
    df: pd.DataFrame, numerical_features: list[str], boolean_features: list[str], 
    categorical_features: list[str], target: str, n_folds: int = 5, use_imputers: bool = False,
):

    def flatten_combo(combo):
        """
        Flattens the combination by including all variables from nested lists
        when one element of a list is selected.
        """
        return list(itertools.chain(*[x if isinstance(x, list) else [x] for x in combo]))

    results_list = []  # To store all results we want for our data frame
    combo_counter = 0  # Initialize a counter for the combinations
    features = numerical_features + boolean_features + categorical_features

    for r in range(1, len(features) + 1):
        for combo in itertools.combinations(features, r):
            
            combo_counter += 1

            # We flatten the list of the resulting combination of features
            flat_combo = flatten_combo(combo)

            # Determine which types of features are present in the current combination
            current_numerical_features = [f for f in flat_combo if f in numerical_features]
            current_boolean_features = [f for f in flat_combo if f in boolean_features]
            current_categorical_features = [f for f in flat_combo if f in categorical_features]

            # Build the transformers dynamically based on the present feature types
            transformers = []
            
            # Add numerical transformer if numerical features are present
            if current_numerical_features:
                numerical_transformer = [('scaler', StandardScaler())]
                if use_imputers:
                    numerical_transformer.append(('median_imputer', SimpleImputer(strategy='median')))
                transformers.append(('num', Pipeline(numerical_transformer), current_numerical_features))
                
            # Add categorical transformer if categorical features are present
            if current_categorical_features:
                categorical_transformer = [('onehot', OneHotEncoder(handle_unknown='error', drop='first'))]
                if use_imputers:
                    categorical_transformer.insert(0, ('simple_imputer', SimpleImputer(strategy='most_frequent')))
                transformers.append(('cat', Pipeline(categorical_transformer), current_categorical_features))
                
            # Add boolean transformer if boolean features are present
            if current_boolean_features:
                if use_imputers:
                    boolean_transformer = [('simple_imputer', SimpleImputer(strategy='most_frequent'))]
                else:
                    boolean_transformer = [('passthrough', 'passthrough')]
                transformers.append(('bool', Pipeline(boolean_transformer), current_boolean_features))
                
            # Define the preprocessor with only the applicable transformers
            preprocessor = ColumnTransformer(transformers=transformers)

            # Pipeline: apply transformations and linear regression
            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('linreg', LinearRegression(fit_intercept=True))
            ])

            # Perform cross-validation and get negative MSE scores
            cv_results = cross_validate(
                estimator=pipeline,
                X=df[flat_combo],
                y=df[target],
                cv=n_folds,
                scoring='neg_mean_squared_error',
                return_estimator=True
            )

            neg_mse = cv_results['test_score']
            estimators = cv_results['estimator']

            # Store results for each fold
            for index, (score, estimator) in enumerate(zip(neg_mse, estimators)):
                linreg = estimator.named_steps['linreg']
                coefs = linreg.coef_
                intercept = linreg.intercept_

                result_dict = {
                    'Fold': index + 1,
                    'Combination ID': combo_counter,
                    'Negative MSE': score,
                    'Intercept': intercept
                }

                # Get feature names post one-hot encoding and exclude original categorical columns
                if current_categorical_features:
                    cat_feature_names = estimator.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(current_categorical_features)
                else:
                    cat_feature_names = []

                # Combine the feature names, excluding the original categorical columns
                combined_features = np.concatenate([current_numerical_features, current_boolean_features, cat_feature_names])

                # Assign coefficients to the corresponding features
                for i, feature in enumerate(combined_features):
                    result_dict[feature] = coefs[i]

                # Assign NaN for features not in the current combination, excluding original categorical columns
                for feature in features:
                    if feature in numerical_features or feature in boolean_features:
                        if feature not in combined_features:
                            result_dict[feature] = float('nan')

                results_list.append(result_dict)

    # Convert results to DataFrame
    results_df = pd.DataFrame(results_list)
    return results_df

### Training with function 8

#### Training with dataset v3

In [59]:
# results_v3_f8 = lin_reg_with_preprocessing_combos_median_imputing(
#     df = df_v3, numerical_features = numerical_cols_v3, boolean_features = boolean_cols_v3,
#     categorical_features = categorical_cols_v3, target = 'price', n_folds = 5,
#     use_imputers = True
# )

# results_v3_f8.to_csv('3_results_datasets/results_v3_f8.csv')

The computation has lasted for approx. 6 minutes.

In [60]:
# results_v3_f8

,Fold,Combination ID,Negative MSE,Intercept,num_rooms,num_baths,square_meters,year_built,num_crimes,is_furnished,...,neighborhood_Sarrià-Sant Gervasi,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9
0,1,1,-72989.051984,1099.686250,5.927694,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,-70145.790556,1101.458125,11.108159,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,-75719.267630,1097.982500,7.062621,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,-74018.007181,1099.884531,9.092078,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,-75968.024362,1099.069844,8.212737,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20470,1,4095,-32452.415571,1057.070134,3.976782,6.839269,192.657693,-0.978987,-76.022831,-8.088217,...,33.492579,44.181298,56.887882,58.365522,72.921340,76.884128,2.398569,4.064126,10.355716,-2.251321
20471,2,4095,-31193.011222,1050.787805,4.288562,7.931738,194.671489,0.483221,-75.230631,-1.776336,...,36.646567,36.163698,53.583105,67.391105,75.046845,74.811854,4.232608,6.053883,5.384287,-1.848843
20472,3,4095,-29785.538590,1046.218061,4.733031,6.125433,189.445293,-1.756328,-76.351799,-1.042517,...,37.915586,44.888069,62.918897,67.505622,69.336506,78.994132,1.945725,6.003837,6.250057,-0.997016
20473,4,4095,-30364.515882,1043.114206,4.717118,9.489858,190.171410,-0.938384,-76.809406,1.862910,...,26.929488,38.164357,58.218733,60.672196,71.115293,76.292979,6.746641,5.543317,8.469710,3.092547


#### Training with dataset v4 (outliers of `num_rooms` treated)

In [10]:
# results_v4_f8 = lin_reg_with_preprocessing_combos_median_imputing(
#     df = df_v4, numerical_features = numerical_cols_v3, boolean_features = boolean_cols_v3,
#     categorical_features = categorical_cols_v3, target = 'price', n_folds = 5,
#     use_imputers = True
# )

# results_v4_f8.to_csv('3_results_datasets/results_v4_f8.csv')

Lasted for 5 mins and 20 seconds.

In [11]:
# results_v4_f8

,Fold,Combination ID,Negative MSE,Intercept,num_rooms,num_baths,square_meters,year_built,num_crimes,is_furnished,...,neighborhood_Sarrià-Sant Gervasi,floor_10,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9
0,1,1,-72645.582001,1099.686250,17.798024,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,-69985.372982,1101.458125,21.319870,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,-75476.784039,1097.982500,18.969422,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,-73962.103178,1099.884531,20.824257,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,-75347.649281,1099.069844,16.536627,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20470,1,4095,-32107.578137,1058.344885,15.600898,6.736999,192.494831,-0.880372,-76.061736,-9.137359,...,33.827971,44.106428,56.952129,58.378583,72.390446,76.886940,2.107064,3.906826,10.273763,-2.538623
20471,2,4095,-31087.107816,1052.209791,17.346073,7.846080,194.500189,0.363171,-74.988043,-3.000922,...,36.868358,37.612424,53.104171,67.612733,76.192771,75.012855,4.304377,5.682957,5.336745,-1.933506
20472,3,4095,-29542.245272,1046.984267,16.520627,6.027764,189.327441,-1.971938,-76.227249,-2.032880,...,39.189093,45.999628,63.210959,67.785116,70.235657,79.155097,1.647823,5.301485,6.397510,-0.536725
20473,4,4095,-30124.080916,1043.414813,16.584430,9.624262,189.962911,-1.355500,-76.526701,0.895805,...,28.268473,39.267347,59.210079,62.230175,72.209402,76.916889,6.449547,5.075136,8.232104,2.946468
